In [ ]:
import sqlalchemy as sqla
import sys
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import subprocess
from plpygis import Geometry
#To avoid pandas warnings
import warnings
warnings.filterwarnings('ignore')
#import geometry_processor_dp
# This function is needed to make the geometries compatible with PostGIS
def wkb_hexer(poly):
    return poly.wkb_hex
from geopandas.tools import sjoin

### Connect to DB

In [ ]:
# Chose the DB
db_input = input ("What database should we use?\n\n") 
# The following data uses data connected at our database

# Create the sqlalchemy engine
db_engine = sqla.create_engine(db_input) #is taken by pd.read_sql_query function later on

# Chose the climate file
cli_input = input("And what climate file?\n\n")  # For now: Martigny
st = time.time()



# Test connection
try:
    db_connection = db_engine.connect()
except Exception as exc:
    print("\nCouldn't connect to the database\n")
    try:
        print(exc.message, exc.args)
    except:
        pass
    sys.exit()
        
# If everything goes well, just be happy about it!
    print("\nHooray! We connected to the database!")

## Download geometry data (2020 database)

In [ ]:
# Download data
print("\nDownloading data...")

buildings_df = gpd.read_postgis("SELECT ST_Transform(ST_SetSRID(ST_Translate(bat.shape,-0.00084328,-0.0012505) ,4326), 21781) AS geometry,"
                                    " COALESCE(rbl.construction_year,1980.1) AS construction_year, bat.shape_area AS coverage, COALESCE(rbl.fake_n_floors,1) AS n_floors, bat.ogr_fid AS ssid,"
                                    " rbl.egid AS egid, COALESCE(NULLIF(rbl.n_pieces,0),1) AS n_people, COALESCE(rbl.fake_n_floors*bat.shape_area*4,1*bat.shape_area*4) AS calc_vol, COALESCE(ocr.occupancytype,1) AS occupancytype "
                                    "FROM satom.batiments bat "
                                    "LEFT JOIN satom.mega mega ON bat.client = mega.megabatiment "
                                    "LEFT JOIN city.occupancy_relation ocr ON ocr.satomtype = mega.fake_megacategorie "
                                    "LEFT JOIN city.regbl_data rbl ON mega.megaegid = rbl.egid "
                                    "JOIN citydb.cross_reference cros on bat.ogr_fid = cros.satom_building_id ", # AND cros.egid = 190056978 ",
                                    #"JOIN citydb.zone2 zone ON cros.egid = zone.egid", # AND cros.egid = 190056978", #need to remove and statement after troubleshoot gmlparentid:  _b45e1df1-4670-4dbc-bf5c-2e548025b419
                                    db_engine, geom_col = 'geometry')

# buildings_df = gpd.read_postgis("SELECT geometry, construction_year, coverage, n_floors, ssid, height, egid, n_people, calc_vol, occupancytype, infiltration_rate FROM city.buildings_egid", db_engine, geom_col='geometry')
print("\nBuilding_df import done...")


In [ ]:
buildings_df = buildings_df.dropna()
# There are 1109 different EGIDs

## Download composite data (2021 database)

In [ ]:
# Chose the DB
db_input_2021 = input ("What database should we use?\n\n") 
# The following data uses data connected at our database

# Create the sqlalchemy engine
db_engine_2021 = sqla.create_engine(db_input_2021) #is taken by pd.read_sql_query function later on

# Test connection
try:
    db_connection = db_engine.connect()
except Exception as exc:
    print("\nCouldn't connect to the database\n")
    try:
        print(exc.message, exc.args)
    except:
        pass
    sys.exit()
        
# If everything goes well, just be happy about it!
    print("\nHooray! We connected to the database!")

In [ ]:
composite_df = pd.read_sql_query("SELECT id, u_value FROM citydb.nrg8_construction", db_engine_2021)
layer_df = pd.read_sql_query("SELECT id, name, pos_nbr, constr_id FROM citydb.nrg8_layer", db_engine_2021)
layer_component_df = pd.read_sql_query("SELECT id, thickness FROM citydb.nrg8_layer_component", db_engine_2021)
material_df = pd.read_sql_query("SELECT id, conductivity, specific_heat, density FROM citydb.nrg8_material", db_engine_2021)
occupancy_df = pd.read_sql_query("SELECT * FROM occupancy_data.occupancy_type", db_engine)

## Geometry envelope

In [ ]:
print("\n Creating envelope...")

if db_engine_2021.has_table('envelope', schema='city'):
    envelope = gpd.read_postgis(
        "SELECT thema.lod2_multi_surface_id AS thema_root_id, install.lod2_brep_id AS lod, surf.geometry AS geometry, thema.objectclass_id AS class_id, REGBL.egid AS egid, surf.cityobject_id AS cityobject_id, construction.constr_id as composite_id " #keep space before quote
        "FROM city.regbl_data REGBL "
        #"FROM citydb.cross_reference cros "
        "LEFT JOIN citydb.cross_reference cros ON cros.egid = REGBL.egid " #remove if doing whole area building_root_id [egid] satom (root_id)
        "JOIN citydb.building build ON build.building_root_id = cros.building_root_id " # [building_root_id] egid satom root_id (building_id objectclass_id building_parent_id)
        #"JOIN citydb.building build ON build.building_root_id = cros.building_root_id AND cros.egid = 190056978 "
        "LEFT JOIN citydb.thematic_surface thema ON thema.building_id = build.id " # building_root_id egid satom root_id [building_id] objectclass_id building_parent_id (thematic_surface_id objeclass_id building_id lod2_multisurface_id)
        "LEFT JOIN citydb.building_installation install ON install.building_id = build.id " # building_root_id egid satom root_id [building_id] objectclass_id building_parent_id thematic_surface_id objeclass_id building_id lod2_multisurface_id (thema.objectclass_id lod2_brep_id)
        "JOIN citydb.surface_geometry surf ON (surf.root_id = thema.lod2_multi_surface_id OR surf.root_id = install.lod2_brep_id) AND surf.geometry IS NOT NULL " # OR surf.root_id = install.lod2_brep_id OR zone.root_id = surf.root_id #remove OR shade.root_id = surf.root_id if doing without shading surfaces
        # building_root_id egid satom root_id building_id objectclass_id building_parent_id thematic_surface_id objeclass_id building_id [lod2_multisurface_id] thema.objectclass_id [lod2_brep_id] (geometry parent_id)
        #"LEFT JOIN city.regbl_data rbl ON rbl.egid = cros.egid "  # building_root_id [egid] satom root_id building_id objectclass_id building_parent_id thematic_surface_id objeclass_id building_id lod2_multisurface_id thema.objectclass_id lod2_brep_id geometry parent_id (construction_period)
        "LEFT JOIN citydb.nrg8_cityobj_to_constr construction ON surf.cityobject_id = construction.cityobject_id",
        #"LEFT JOIN surfaces_data.default_composites comp ON comp.class_id = thema.objectclass_id AND rbl.construction_year BETWEEN comp.period_start AND comp.period_end",
        db_engine_2021, geom_col='geometry')
        
db_connection.close()

# Scale if needed
envelope_box = envelope.geometry.total_bounds
mean_x = (envelope_box[2] + envelope_box[0]) / 2
mean_y = (envelope_box[3] + envelope_box[1]) / 2
if mean_x > 10000 or mean_y > 10000:
    envelope.geometry = envelope.geometry.translate(xoff=-mean_x, yoff=-mean_y, zoff=0.0)

else:
    # Create the table if it doesn't exist
    print("Creating surfaces...")
    db_connection.close()
    # Scale if needed
    city_box = buildings_df.geometry.total_bounds
    mean_x = (city_box[2] + city_box[0])/2
    mean_y = (city_box[3] + city_box[1])/2

In [ ]:
envelope = envelope.dropna(subset=['composite_id'])
envelope = envelope.dropna(subset=['geometry'])
envelope = envelope.drop_duplicates(subset=['geometry'])

In [ ]:
#Translate the geometry to 0,0 so that there are not too many significant digits in CitySim

x_max = envelope.total_bounds[2]
y_max = envelope.total_bounds[3]
envelope["geometry"] = envelope.translate(xoff=-x_max, yoff=-y_max) #take coord from total bounds
envelope

# Geometry filtration : keeping geometries that contain at least one of each class type (roof, floor, wall)


In [ ]:


egid_list = envelope.drop(["thema_root_id", "lod", "geometry", "cityobject_id", "composite_id"], axis=1)
egid_list = egid_list.drop_duplicates(subset=['egid','class_id'])

egid_list = egid_list.groupby(['egid']).sum()
egid_list
egid_list = egid_list[egid_list['class_id'] == 102]
egid_list = egid_list.drop(['class_id'], axis=1)
egid_list

final_envelope = pd.merge(egid_list, envelope, how='left', left_on=['egid'], right_on=['egid'])
final_envelope



# Geometry shapefile for visual verification 

In [ ]:
final_envelope = gpd.GeoDataFrame(final_envelope)

In [ ]:
final_envelope.to_file("final_envelope.shp")

# Finalising buildings_df so it contains only the same egid as final_envelope

In [ ]:
# List of egid from buildings_df
egid_to_keep = final_envelope["egid"].dropna().drop_duplicates()
buildings_df = buildings_df[buildings_df['egid'].isin(egid_to_keep)]

In [ ]:
buildings_df = buildings_df.drop_duplicates(subset=["egid"])
buildings_df

## Create/modify xml file

In [ ]:
# Create/overwrite the XML file, standard CitySim header
print("\nCreating the XML file...")
xml_name = str(cli_input) + "_to_3D.xml"
name = str(cli_input) + "_to_3D"
print('<?xml version="1.0" encoding="ISO-8859-1"?>\n<CitySim>', file=open(xml_name, "w"))

# Define the simulation period
print('<Simulation beginMonth="1" endMonth="12" beginDay="1" endDay="31"/>', file=open(xml_name, "a"))

# Define the climate file
print('<Climate location="' + str(cli_input) + '.cli"/>', file=open(xml_name, "a"))

# Open the "District" tab
print('<District>', file=open(xml_name, "a"))

"""
# Define far field obstructions
print('<FarFieldObstructions>', file=open(xml_name, "a"))
for r in horizon_df.index:
    print('<Point phi=\"' + str(horizon_df['phi'].loc[r]) + '\"  theta=\"' + str(horizon_df['theta'].loc[r]) + '\"/>', file=open(xml_name, "a"))
    print('</FarFieldObstructions>', file=open(xml_name, "a"))
    """

## Reconstructing composites data 

In [ ]:
# Define composites
lay_df = pd.merge(layer_df, layer_component_df, how='left', left_on=['id'], right_on=['id'])
final_df = pd.merge(lay_df, material_df, how='left', left_on=['id'], right_on=['id'])


for r in composite_df.index: #technique python pour parcourir un string
    comp_id = composite_df["id"].loc[r] #take value of composite id
    uval = composite_df["u_value"].loc[r]
    layermat_df = final_df.loc[final_df["constr_id"] == comp_id].sort_values(by=["pos_nbr"])
    #print('<Composite id="' + str(comp_id) + '">', file=open(xml_name, "a"))
    
    if not layermat_df.empty: #If there are layers
        print('<Composite id="' + str(comp_id) + '" Uvalue="'+str(uval)+'">', file=open(xml_name, "a"))
        
        
        for r1 in layermat_df.index:
            layer_name = layermat_df["name"].loc[r1]
            thickness = layermat_df["thickness"].loc[r1]
            conductivity = layermat_df["conductivity"].loc[r1]
            cp = layermat_df["specific_heat"].loc[r1]
            density = layermat_df["density"].loc[r1]
            print('<Layer Name="' + layer_name + '" Thickness="' +
                      str(thickness) + '" Conductivity="' +
                      str(conductivity) + '" Cp="' + str(cp) +
                      '" Density="' + str(density) + '"/>', file=open(xml_name, "a"))
            
        else:
            pass
        print('</Composite>', file=open(xml_name, "a"))
    
    else: # if no layers
        print('<Composite id="' + str(comp_id) + '" Uvalue="'+str(uval)+'"/>', file=open(xml_name, "a"))
        
        

## Defining occupancies, profiles...

In [ ]:
# Define occupancy daily/yearly profiles
print(
        '<OccupancyDayProfile id="0" p1="0.0" p2="0.0" p3="0.0" p4="0.0" p5="0.0" p6="0.0" p7="0.0" p8="0.0" p9="0.0" p10="0.0" p11="0.0" p12="0.0" p13="0.0" p14="0.0" p15="0.0" p16="0.0" p17="0.0" p18="0.0" p19="0.0" p20="0.0" p21="0.0" p22="0.0" p23="0.0" p24="0.0"/>\n' +
        '<OccupancyDayProfile id="1" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="0.8" p8="0.6" p9="0.4" p10="0.4" p11="0.4" p12="0.6" p13="0.8" p14="0.6" p15="0.4" p16="0.4" p17="0.6" p18="0.8" p19="0.8" p20="0.8" p21="0.8" p22="1.0" p23="1.0" p24="1.0"/>\n' +
        '<OccupancyDayProfile id="2" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.6" p11="0.8" p12="0.8" p13="0.4" p14="0.6" p15="0.8" p16="0.8" p17="0.4" p18="0.2" p19="0.001" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
        '<OccupancyDayProfile id="3" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.6" p10="1.0" p11="1.0" p12="0.2" p13="0.2" p14="1.0" p15="1.0" p16="0.6" p17="0.6" p18="0.6" p19="0.6" p20="1.0" p21="1.0" p22="0.6" p23="0.2" p24="0.001"/>\n' +
        '<OccupancyDayProfile id="4" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.4" p11="0.4" p12="0.6" p13="0.6" p14="0.6" p15="0.4" p16="0.4" p17="0.6" p18="0.8" p19="0.6" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
        '<OccupancyDayProfile id="5" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.001" p9="0.2" p10="0.2" p11="0.2" p12="0.2" p13="0.8" p14="0.4" p15="0.001" p16="0.001" p17="0.001" p18="0.001" p19="0.2" p20="0.2" p21="0.4" p22="0.8" p23="0.2" p24="0.2"/>\n' +
        '<OccupancyDayProfile id="6" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="0.8" p8="0.4" p9="0.001" p10="0.001" p11="0.001" p12="0.001" p13="0.001" p14="0.001" p15="0.001" p16="0.001" p17="0.001" p18="0.001" p19="0.001" p20="0.001" p21="0.6" p22="0.6" p23="0.8" p24="0.6"/>\n' +
        '<OccupancyDayProfile id="7" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0"/>\n' +
        '<OccupancyDayProfile id="8" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.8" p11="0.8" p12="0.4" p13="0.2" p14="0.4" p15="0.8" p16="0.8" p17="0.2" p18="0.001" p19="0.001" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
        '<OccupancyDayProfile id="9" p1="0.2" p2="0.2" p3="0.2" p4="0.2" p5="0.2" p6="0.5" p7="0.8" p8="1.0" p9="1.0" p10="0.8" p11="1.0" p12="0.4" p13="0.6" p14="1.0" p15="1.0" p16="0.8" p17="0.8" p18="0.8" p19="0.4" p20="0.4" p21="0.4" p22="0.4" p23="0.2" p24="0.2"/>\n' +
        '<OccupancyDayProfile id="10" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.6" p11="0.8" p12="0.8" p13="0.4" p14="0.6" p15="0.8" p16="0.8" p17="0.4" p18="0.2" p19="0.001" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
        '<OccupancyYearProfile id="1" d1="1" d2="1" d3="1" d4="1" d5="1" d6="1" d7="1" d8="1" d9="1" d10="1" d11="1" d12="1" d13="1" d14="1" d15="1" d16="1" d17="1" d18="1" d19="1" d20="1" d21="1" d22="1" d23="1" d24="1" d25="1" d26="1" d27="1" d28="1" d29="1" d30="1" d31="1" d32="1" d33="1" d34="1" d35="1" d36="1" d37="1" d38="1" d39="1" d40="1" d41="1" d42="1" d43="1" d44="1" d45="1" d46="1" d47="1" d48="1" d49="1" d50="1" d51="1" d52="1" d53="1" d54="1" d55="1" d56="1" d57="1" d58="1" d59="1" d60="1" d61="1" d62="1" d63="1" d64="1" d65="1" d66="1" d67="1" d68="1" d69="1" d70="1" d71="1" d72="1" d73="1" d74="1" d75="1" d76="1" d77="1" d78="1" d79="1" d80="1" d81="1" d82="1" d83="1" d84="1" d85="1" d86="1" d87="1" d88="1" d89="1" d90="1" d91="1" d92="1" d93="1" d94="1" d95="1" d96="1" d97="1" d98="1" d99="1" d100="1" d101="1" d102="1" d103="1" d104="1" d105="1" d106="1" d107="1" d108="1" d109="1" d110="1" d111="1" d112="1" d113="1" d114="1" d115="1" d116="1" d117="1" d118="1" d119="1" d120="1" d121="1" d122="1" d123="1" d124="1" d125="1" d126="1" d127="1" d128="1" d129="1" d130="1" d131="1" d132="1" d133="1" d134="1" d135="1" d136="1" d137="1" d138="1" d139="1" d140="1" d141="1" d142="1" d143="1" d144="1" d145="1" d146="1" d147="1" d148="1" d149="1" d150="1" d151="1" d152="1" d153="1" d154="1" d155="1" d156="1" d157="1" d158="1" d159="1" d160="1" d161="1" d162="1" d163="1" d164="1" d165="1" d166="1" d167="1" d168="1" d169="1" d170="1" d171="1" d172="1" d173="1" d174="1" d175="1" d176="1" d177="1" d178="1" d179="1" d180="1" d181="1" d182="1" d183="1" d184="1" d185="1" d186="1" d187="1" d188="1" d189="1" d190="1" d191="1" d192="1" d193="1" d194="1" d195="1" d196="1" d197="1" d198="1" d199="1" d200="1" d201="1" d202="1" d203="1" d204="1" d205="1" d206="1" d207="1" d208="1" d209="1" d210="1" d211="1" d212="1" d213="1" d214="1" d215="1" d216="1" d217="1" d218="1" d219="1" d220="1" d221="1" d222="1" d223="1" d224="1" d225="1" d226="1" d227="1" d228="1" d229="1" d230="1" d231="1" d232="1" d233="1" d234="1" d235="1" d236="1" d237="1" d238="1" d239="1" d240="1" d241="1" d242="1" d243="1" d244="1" d245="1" d246="1" d247="1" d248="1" d249="1" d250="1" d251="1" d252="1" d253="1" d254="1" d255="1" d256="1" d257="1" d258="1" d259="1" d260="1" d261="1" d262="1" d263="1" d264="1" d265="1" d266="1" d267="1" d268="1" d269="1" d270="1" d271="1" d272="1" d273="1" d274="1" d275="1" d276="1" d277="1" d278="1" d279="1" d280="1" d281="1" d282="1" d283="1" d284="1" d285="1" d286="1" d287="1" d288="1" d289="1" d290="1" d291="1" d292="1" d293="1" d294="1" d295="1" d296="1" d297="1" d298="1" d299="1" d300="1" d301="1" d302="1" d303="1" d304="1" d305="1" d306="1" d307="1" d308="1" d309="1" d310="1" d311="1" d312="1" d313="1" d314="1" d315="1" d316="1" d317="1" d318="1" d319="1" d320="1" d321="1" d322="1" d323="1" d324="1" d325="1" d326="1" d327="1" d328="1" d329="1" d330="1" d331="1" d332="1" d333="1" d334="1" d335="1" d336="1" d337="1" d338="1" d339="1" d340="1" d341="1" d342="1" d343="1" d344="1" d345="1" d346="1" d347="1" d348="1" d349="1" d350="1" d351="1" d352="1" d353="1" d354="1" d355="1" d356="1" d357="1" d358="1" d359="1" d360="1" d361="1" d362="1" d363="1" d364="1" d365="1"/>\n' +
        '<OccupancyYearProfile id="2" d1="2" d2="2" d3="2" d4="2" d5="2" d6="0" d7="0" d8="2" d9="2" d10="2" d11="2" d12="2" d13="0" d14="0" d15="2" d16="2" d17="2" d18="2" d19="2" d20="0" d21="0" d22="2" d23="2" d24="2" d25="2" d26="2" d27="0" d28="0" d29="2" d30="2" d31="2" d32="2" d33="2" d34="0" d35="0" d36="2" d37="2" d38="2" d39="2" d40="2" d41="0" d42="0" d43="2" d44="2" d45="2" d46="2" d47="2" d48="0" d49="0" d50="2" d51="2" d52="2" d53="2" d54="2" d55="0" d56="0" d57="2" d58="2" d59="2" d60="2" d61="2" d62="0" d63="0" d64="2" d65="2" d66="2" d67="2" d68="2" d69="0" d70="0" d71="2" d72="2" d73="2" d74="2" d75="2" d76="0" d77="0" d78="2" d79="2" d80="2" d81="2" d82="2" d83="0" d84="0" d85="2" d86="2" d87="2" d88="2" d89="2" d90="0" d91="0" d92="2" d93="2" d94="2" d95="2" d96="2" d97="0" d98="0" d99="2" d100="2" d101="2" d102="2" d103="2" d104="0" d105="0" d106="2" d107="2" d108="2" d109="2" d110="2" d111="0" d112="0" d113="2" d114="2" d115="2" d116="2" d117="2" d118="0" d119="0" d120="2" d121="2" d122="2" d123="2" d124="2" d125="0" d126="0" d127="2" d128="2" d129="2" d130="2" d131="2" d132="0" d133="0" d134="2" d135="2" d136="2" d137="2" d138="2" d139="0" d140="0" d141="2" d142="2" d143="2" d144="2" d145="2" d146="0" d147="0" d148="2" d149="2" d150="2" d151="2" d152="2" d153="0" d154="0" d155="2" d156="2" d157="2" d158="2" d159="2" d160="0" d161="0" d162="2" d163="2" d164="2" d165="2" d166="2" d167="0" d168="0" d169="2" d170="2" d171="2" d172="2" d173="2" d174="0" d175="0" d176="2" d177="2" d178="2" d179="2" d180="2" d181="0" d182="0" d183="2" d184="2" d185="2" d186="2" d187="2" d188="0" d189="0" d190="2" d191="2" d192="2" d193="2" d194="2" d195="0" d196="0" d197="2" d198="2" d199="2" d200="2" d201="2" d202="0" d203="0" d204="2" d205="2" d206="2" d207="2" d208="2" d209="0" d210="0" d211="2" d212="2" d213="2" d214="2" d215="2" d216="0" d217="0" d218="2" d219="2" d220="2" d221="2" d222="2" d223="0" d224="0" d225="2" d226="2" d227="2" d228="2" d229="2" d230="0" d231="0" d232="2" d233="2" d234="2" d235="2" d236="2" d237="0" d238="0" d239="2" d240="2" d241="2" d242="2" d243="2" d244="0" d245="0" d246="2" d247="2" d248="2" d249="2" d250="2" d251="0" d252="0" d253="2" d254="2" d255="2" d256="2" d257="2" d258="0" d259="0" d260="2" d261="2" d262="2" d263="2" d264="2" d265="2" d266="0" d267="0" d268="2" d269="2" d270="2" d271="2" d272="2" d273="0" d274="0" d275="2" d276="2" d277="2" d278="2" d279="2" d280="0" d281="0" d282="2" d283="2" d284="2" d285="2" d286="2" d287="0" d288="0" d289="2" d290="2" d291="2" d292="2" d293="2" d294="0" d295="0" d296="2" d297="2" d298="2" d299="2" d300="2" d301="0" d302="0" d303="2" d304="2" d305="2" d306="2" d307="2" d308="0" d309="0" d310="2" d311="2" d312="2" d313="2" d314="2" d315="0" d316="0" d317="2" d318="2" d319="2" d320="2" d321="2" d322="0" d323="0" d324="2" d325="2" d326="2" d327="2" d328="2" d329="0" d330="0" d331="2" d332="2" d333="2" d334="2" d335="2" d336="0" d337="0" d338="2" d339="2" d340="2" d341="2" d342="2" d343="0" d344="0" d345="2" d346="2" d347="2" d348="2" d349="2" d350="0" d351="0" d352="2" d353="2" d354="2" d355="2" d356="2" d357="0" d358="0" d359="2" d360="2" d361="2" d362="2" d363="2" d364="0" d365="0"/>\n' +
        '<OccupancyYearProfile id="3" d1="3" d2="3" d3="3" d4="3" d5="3" d6="3" d7="3" d8="3" d9="3" d10="3" d11="3" d12="3" d13="3" d14="3" d15="3" d16="3" d17="3" d18="3" d19="3" d20="3" d21="3" d22="3" d23="3" d24="3" d25="3" d26="3" d27="3" d28="3" d29="3" d30="3" d31="3" d32="3" d33="3" d34="3" d35="3" d36="3" d37="3" d38="3" d39="3" d40="3" d41="3" d42="3" d43="3" d44="3" d45="3" d46="3" d47="3" d48="3" d49="3" d50="3" d51="3" d52="3" d53="3" d54="3" d55="3" d56="3" d57="3" d58="3" d59="3" d60="3" d61="3" d62="3" d63="3" d64="3" d65="3" d66="3" d67="3" d68="3" d69="3" d70="3" d71="3" d72="3" d73="3" d74="3" d75="3" d76="3" d77="3" d78="3" d79="3" d80="3" d81="3" d82="3" d83="3" d84="3" d85="3" d86="3" d87="3" d88="3" d89="3" d90="3" d91="3" d92="3" d93="3" d94="3" d95="3" d96="3" d97="3" d98="3" d99="3" d100="3" d101="3" d102="3" d103="3" d104="3" d105="3" d106="3" d107="3" d108="3" d109="3" d110="3" d111="3" d112="3" d113="3" d114="3" d115="3" d116="3" d117="3" d118="3" d119="3" d120="3" d121="3" d122="3" d123="3" d124="3" d125="3" d126="3" d127="3" d128="3" d129="3" d130="3" d131="3" d132="3" d133="3" d134="3" d135="3" d136="3" d137="3" d138="3" d139="3" d140="3" d141="3" d142="3" d143="3" d144="3" d145="3" d146="3" d147="3" d148="3" d149="3" d150="3" d151="3" d152="3" d153="3" d154="3" d155="3" d156="3" d157="3" d158="3" d159="3" d160="3" d161="3" d162="3" d163="3" d164="3" d165="3" d166="3" d167="3" d168="3" d169="3" d170="3" d171="3" d172="3" d173="3" d174="3" d175="3" d176="3" d177="3" d178="3" d179="3" d180="3" d181="3" d182="3" d183="3" d184="3" d185="3" d186="3" d187="3" d188="3" d189="3" d190="3" d191="3" d192="3" d193="3" d194="3" d195="3" d196="3" d197="3" d198="3" d199="3" d200="3" d201="3" d202="3" d203="3" d204="3" d205="3" d206="3" d207="3" d208="3" d209="3" d210="3" d211="3" d212="3" d213="3" d214="3" d215="3" d216="3" d217="3" d218="3" d219="3" d220="3" d221="3" d222="3" d223="3" d224="3" d225="3" d226="3" d227="3" d228="3" d229="3" d230="3" d231="3" d232="3" d233="3" d234="3" d235="3" d236="3" d237="3" d238="3" d239="3" d240="3" d241="3" d242="3" d243="3" d244="3" d245="3" d246="3" d247="3" d248="3" d249="3" d250="3" d251="3" d252="3" d253="3" d254="3" d255="3" d256="3" d257="3" d258="3" d259="3" d260="3" d261="3" d262="3" d263="3" d264="3" d265="3" d266="3" d267="3" d268="3" d269="3" d270="3" d271="3" d272="3" d273="3" d274="3" d275="3" d276="3" d277="3" d278="3" d279="3" d280="3" d281="3" d282="3" d283="3" d284="3" d285="3" d286="3" d287="3" d288="3" d289="3" d290="3" d291="3" d292="3" d293="3" d294="3" d295="3" d296="3" d297="3" d298="3" d299="3" d300="3" d301="3" d302="3" d303="3" d304="3" d305="3" d306="3" d307="3" d308="3" d309="3" d310="3" d311="3" d312="3" d313="3" d314="3" d315="3" d316="3" d317="3" d318="3" d319="3" d320="3" d321="3" d322="3" d323="3" d324="3" d325="3" d326="3" d327="3" d328="3" d329="3" d330="3" d331="3" d332="3" d333="3" d334="3" d335="3" d336="3" d337="3" d338="3" d339="3" d340="3" d341="3" d342="3" d343="3" d344="3" d345="3" d346="3" d347="3" d348="3" d349="3" d350="3" d351="3" d352="3" d353="3" d354="3" d355="3" d356="3" d357="3" d358="3" d359="3" d360="3" d361="3" d362="3" d363="3" d364="3" d365="3"/>\n' +
        '<OccupancyYearProfile id="4" d1="4" d2="4" d3="4" d4="4" d5="4" d6="4" d7="4" d8="4" d9="4" d10="4" d11="4" d12="4" d13="4" d14="4" d15="4" d16="4" d17="4" d18="4" d19="4" d20="4" d21="4" d22="4" d23="4" d24="4" d25="4" d26="4" d27="4" d28="4" d29="4" d30="4" d31="4" d32="4" d33="4" d34="4" d35="4" d36="4" d37="4" d38="4" d39="4" d40="4" d41="4" d42="4" d43="4" d44="4" d45="4" d46="4" d47="4" d48="4" d49="4" d50="4" d51="4" d52="4" d53="4" d54="4" d55="4" d56="4" d57="4" d58="4" d59="4" d60="4" d61="4" d62="4" d63="4" d64="4" d65="4" d66="4" d67="4" d68="4" d69="4" d70="4" d71="4" d72="4" d73="4" d74="4" d75="4" d76="4" d77="4" d78="4" d79="4" d80="4" d81="4" d82="4" d83="4" d84="4" d85="4" d86="4" d87="4" d88="4" d89="4" d90="4" d91="4" d92="4" d93="4" d94="4" d95="4" d96="4" d97="4" d98="4" d99="4" d100="4" d101="4" d102="4" d103="4" d104="4" d105="4" d106="4" d107="4" d108="4" d109="4" d110="4" d111="4" d112="4" d113="4" d114="4" d115="4" d116="4" d117="4" d118="4" d119="4" d120="4" d121="4" d122="4" d123="4" d124="4" d125="4" d126="4" d127="4" d128="4" d129="4" d130="4" d131="4" d132="4" d133="4" d134="4" d135="4" d136="4" d137="4" d138="4" d139="4" d140="4" d141="4" d142="4" d143="4" d144="4" d145="4" d146="4" d147="4" d148="4" d149="4" d150="4" d151="4" d152="4" d153="4" d154="4" d155="4" d156="4" d157="4" d158="4" d159="4" d160="4" d161="4" d162="4" d163="4" d164="4" d165="4" d166="4" d167="4" d168="4" d169="4" d170="4" d171="4" d172="4" d173="4" d174="4" d175="4" d176="4" d177="4" d178="4" d179="4" d180="4" d181="4" d182="4" d183="4" d184="4" d185="4" d186="4" d187="4" d188="4" d189="4" d190="4" d191="4" d192="4" d193="4" d194="4" d195="4" d196="4" d197="4" d198="4" d199="4" d200="4" d201="4" d202="4" d203="4" d204="4" d205="4" d206="4" d207="4" d208="4" d209="4" d210="4" d211="4" d212="4" d213="4" d214="4" d215="4" d216="4" d217="4" d218="4" d219="4" d220="4" d221="4" d222="4" d223="4" d224="4" d225="4" d226="4" d227="4" d228="4" d229="4" d230="4" d231="4" d232="4" d233="4" d234="4" d235="4" d236="4" d237="4" d238="4" d239="4" d240="4" d241="4" d242="4" d243="4" d244="4" d245="4" d246="4" d247="4" d248="4" d249="4" d250="4" d251="4" d252="4" d253="4" d254="4" d255="4" d256="4" d257="4" d258="4" d259="4" d260="4" d261="4" d262="4" d263="4" d264="4" d265="4" d266="4" d267="4" d268="4" d269="4" d270="4" d271="4" d272="4" d273="4" d274="4" d275="4" d276="4" d277="4" d278="4" d279="4" d280="4" d281="4" d282="4" d283="4" d284="4" d285="4" d286="4" d287="4" d288="4" d289="4" d290="4" d291="4" d292="4" d293="4" d294="4" d295="4" d296="4" d297="4" d298="4" d299="4" d300="4" d301="4" d302="4" d303="4" d304="4" d305="4" d306="4" d307="4" d308="4" d309="4" d310="4" d311="4" d312="4" d313="4" d314="4" d315="4" d316="4" d317="4" d318="4" d319="4" d320="4" d321="4" d322="4" d323="4" d324="4" d325="4" d326="4" d327="4" d328="4" d329="4" d330="4" d331="4" d332="4" d333="4" d334="4" d335="4" d336="4" d337="4" d338="4" d339="4" d340="4" d341="4" d342="4" d343="4" d344="4" d345="4" d346="4" d347="4" d348="4" d349="4" d350="4" d351="4" d352="4" d353="4" d354="4" d355="4" d356="4" d357="4" d358="4" d359="4" d360="4" d361="4" d362="4" d363="4" d364="4" d365="4"/>\n' +
        '<OccupancyYearProfile id="5" d1="5" d2="5" d3="5" d4="5" d5="5" d6="5" d7="5" d8="5" d9="5" d10="5" d11="5" d12="5" d13="5" d14="5" d15="5" d16="5" d17="5" d18="5" d19="5" d20="5" d21="5" d22="5" d23="5" d24="5" d25="5" d26="5" d27="5" d28="5" d29="5" d30="5" d31="5" d32="5" d33="5" d34="5" d35="5" d36="5" d37="5" d38="5" d39="5" d40="5" d41="5" d42="5" d43="5" d44="5" d45="5" d46="5" d47="5" d48="5" d49="5" d50="5" d51="5" d52="5" d53="5" d54="5" d55="5" d56="5" d57="5" d58="5" d59="5" d60="5" d61="5" d62="5" d63="5" d64="5" d65="5" d66="5" d67="5" d68="5" d69="5" d70="5" d71="5" d72="5" d73="5" d74="5" d75="5" d76="5" d77="5" d78="5" d79="5" d80="5" d81="5" d82="5" d83="5" d84="5" d85="5" d86="5" d87="5" d88="5" d89="5" d90="5" d91="5" d92="5" d93="5" d94="5" d95="5" d96="5" d97="5" d98="5" d99="5" d100="5" d101="5" d102="5" d103="5" d104="5" d105="5" d106="5" d107="5" d108="5" d109="5" d110="5" d111="5" d112="5" d113="5" d114="5" d115="5" d116="5" d117="5" d118="5" d119="5" d120="5" d121="5" d122="5" d123="5" d124="5" d125="5" d126="5" d127="5" d128="5" d129="5" d130="5" d131="5" d132="5" d133="5" d134="5" d135="5" d136="5" d137="5" d138="5" d139="5" d140="5" d141="5" d142="5" d143="5" d144="5" d145="5" d146="5" d147="5" d148="5" d149="5" d150="5" d151="5" d152="5" d153="5" d154="5" d155="5" d156="5" d157="5" d158="5" d159="5" d160="5" d161="5" d162="5" d163="5" d164="5" d165="5" d166="5" d167="5" d168="5" d169="5" d170="5" d171="5" d172="5" d173="5" d174="5" d175="5" d176="5" d177="5" d178="5" d179="5" d180="5" d181="5" d182="5" d183="5" d184="5" d185="5" d186="5" d187="5" d188="5" d189="5" d190="5" d191="5" d192="5" d193="5" d194="5" d195="5" d196="5" d197="5" d198="5" d199="5" d200="5" d201="5" d202="5" d203="5" d204="5" d205="5" d206="5" d207="5" d208="5" d209="5" d210="5" d211="5" d212="5" d213="5" d214="5" d215="5" d216="5" d217="5" d218="5" d219="5" d220="5" d221="5" d222="5" d223="5" d224="5" d225="5" d226="5" d227="5" d228="5" d229="5" d230="5" d231="5" d232="5" d233="5" d234="5" d235="5" d236="5" d237="5" d238="5" d239="5" d240="5" d241="5" d242="5" d243="5" d244="5" d245="5" d246="5" d247="5" d248="5" d249="5" d250="5" d251="5" d252="5" d253="5" d254="5" d255="5" d256="5" d257="5" d258="5" d259="5" d260="5" d261="5" d262="5" d263="5" d264="5" d265="5" d266="5" d267="5" d268="5" d269="5" d270="5" d271="5" d272="5" d273="5" d274="5" d275="5" d276="5" d277="5" d278="5" d279="5" d280="5" d281="5" d282="5" d283="5" d284="5" d285="5" d286="5" d287="5" d288="5" d289="5" d290="5" d291="5" d292="5" d293="5" d294="5" d295="5" d296="5" d297="5" d298="5" d299="5" d300="5" d301="5" d302="5" d303="5" d304="5" d305="5" d306="5" d307="5" d308="5" d309="5" d310="5" d311="5" d312="5" d313="5" d314="5" d315="5" d316="5" d317="5" d318="5" d319="5" d320="5" d321="5" d322="5" d323="5" d324="5" d325="5" d326="5" d327="5" d328="5" d329="5" d330="5" d331="5" d332="5" d333="5" d334="5" d335="5" d336="5" d337="5" d338="5" d339="5" d340="5" d341="5" d342="5" d343="5" d344="5" d345="5" d346="5" d347="5" d348="5" d349="5" d350="5" d351="5" d352="5" d353="5" d354="5" d355="5" d356="5" d357="5" d358="5" d359="5" d360="5" d361="5" d362="5" d363="5" d364="5" d365="5"/>\n' +
        '<OccupancyYearProfile id="6" d1="6" d2="6" d3="6" d4="6" d5="6" d6="6" d7="6" d8="6" d9="6" d10="6" d11="6" d12="6" d13="6" d14="6" d15="6" d16="6" d17="6" d18="6" d19="6" d20="6" d21="6" d22="6" d23="6" d24="6" d25="6" d26="6" d27="6" d28="6" d29="6" d30="6" d31="6" d32="6" d33="6" d34="6" d35="6" d36="6" d37="6" d38="6" d39="6" d40="6" d41="6" d42="6" d43="6" d44="6" d45="6" d46="6" d47="6" d48="6" d49="6" d50="6" d51="6" d52="6" d53="6" d54="6" d55="6" d56="6" d57="6" d58="6" d59="6" d60="6" d61="6" d62="6" d63="6" d64="6" d65="6" d66="6" d67="6" d68="6" d69="6" d70="6" d71="6" d72="6" d73="6" d74="6" d75="6" d76="6" d77="6" d78="6" d79="6" d80="6" d81="6" d82="6" d83="6" d84="6" d85="6" d86="6" d87="6" d88="6" d89="6" d90="6" d91="6" d92="6" d93="6" d94="6" d95="6" d96="6" d97="6" d98="6" d99="6" d100="6" d101="6" d102="6" d103="6" d104="6" d105="6" d106="6" d107="6" d108="6" d109="6" d110="6" d111="6" d112="6" d113="6" d114="6" d115="6" d116="6" d117="6" d118="6" d119="6" d120="6" d121="6" d122="6" d123="6" d124="6" d125="6" d126="6" d127="6" d128="6" d129="6" d130="6" d131="6" d132="6" d133="6" d134="6" d135="6" d136="6" d137="6" d138="6" d139="6" d140="6" d141="6" d142="6" d143="6" d144="6" d145="6" d146="6" d147="6" d148="6" d149="6" d150="6" d151="6" d152="6" d153="6" d154="6" d155="6" d156="6" d157="6" d158="6" d159="6" d160="6" d161="6" d162="6" d163="6" d164="6" d165="6" d166="6" d167="6" d168="6" d169="6" d170="6" d171="6" d172="6" d173="6" d174="6" d175="6" d176="6" d177="6" d178="6" d179="6" d180="6" d181="6" d182="6" d183="6" d184="6" d185="6" d186="6" d187="6" d188="6" d189="6" d190="6" d191="6" d192="6" d193="6" d194="6" d195="6" d196="6" d197="6" d198="6" d199="6" d200="6" d201="6" d202="6" d203="6" d204="6" d205="6" d206="6" d207="6" d208="6" d209="6" d210="6" d211="6" d212="6" d213="6" d214="6" d215="6" d216="6" d217="6" d218="6" d219="6" d220="6" d221="6" d222="6" d223="6" d224="6" d225="6" d226="6" d227="6" d228="6" d229="6" d230="6" d231="6" d232="6" d233="6" d234="6" d235="6" d236="6" d237="6" d238="6" d239="6" d240="6" d241="6" d242="6" d243="6" d244="6" d245="6" d246="6" d247="6" d248="6" d249="6" d250="6" d251="6" d252="6" d253="6" d254="6" d255="6" d256="6" d257="6" d258="6" d259="6" d260="6" d261="6" d262="6" d263="6" d264="6" d265="6" d266="6" d267="6" d268="6" d269="6" d270="6" d271="6" d272="6" d273="6" d274="6" d275="6" d276="6" d277="6" d278="6" d279="6" d280="6" d281="6" d282="6" d283="6" d284="6" d285="6" d286="6" d287="6" d288="6" d289="6" d290="6" d291="6" d292="6" d293="6" d294="6" d295="6" d296="6" d297="6" d298="6" d299="6" d300="6" d301="6" d302="6" d303="6" d304="6" d305="6" d306="6" d307="6" d308="6" d309="6" d310="6" d311="6" d312="6" d313="6" d314="6" d315="6" d316="6" d317="6" d318="6" d319="6" d320="6" d321="6" d322="6" d323="6" d324="6" d325="6" d326="6" d327="6" d328="6" d329="6" d330="6" d331="6" d332="6" d333="6" d334="6" d335="6" d336="6" d337="6" d338="6" d339="6" d340="6" d341="6" d342="6" d343="6" d344="6" d345="6" d346="6" d347="6" d348="6" d349="6" d350="6" d351="6" d352="6" d353="6" d354="6" d355="6" d356="6" d357="6" d358="6" d359="6" d360="6" d361="6" d362="6" d363="6" d364="6" d365="6"/>\n' +
        '<OccupancyYearProfile id="7" d1="7" d2="7" d3="7" d4="7" d5="7" d6="7" d7="7" d8="7" d9="7" d10="7" d11="7" d12="7" d13="7" d14="7" d15="7" d16="7" d17="7" d18="7" d19="7" d20="7" d21="7" d22="7" d23="7" d24="7" d25="7" d26="7" d27="7" d28="7" d29="7" d30="7" d31="7" d32="7" d33="7" d34="7" d35="7" d36="7" d37="7" d38="7" d39="7" d40="7" d41="7" d42="7" d43="7" d44="7" d45="7" d46="7" d47="7" d48="7" d49="7" d50="7" d51="7" d52="7" d53="7" d54="7" d55="7" d56="7" d57="7" d58="7" d59="7" d60="7" d61="7" d62="7" d63="7" d64="7" d65="7" d66="7" d67="7" d68="7" d69="7" d70="7" d71="7" d72="7" d73="7" d74="7" d75="7" d76="7" d77="7" d78="7" d79="7" d80="7" d81="7" d82="7" d83="7" d84="7" d85="7" d86="7" d87="7" d88="7" d89="7" d90="7" d91="7" d92="7" d93="7" d94="7" d95="7" d96="7" d97="7" d98="7" d99="7" d100="7" d101="7" d102="7" d103="7" d104="7" d105="7" d106="7" d107="7" d108="7" d109="7" d110="7" d111="7" d112="7" d113="7" d114="7" d115="7" d116="7" d117="7" d118="7" d119="7" d120="7" d121="7" d122="7" d123="7" d124="7" d125="7" d126="7" d127="7" d128="7" d129="7" d130="7" d131="7" d132="7" d133="7" d134="7" d135="7" d136="7" d137="7" d138="7" d139="7" d140="7" d141="7" d142="7" d143="7" d144="7" d145="7" d146="7" d147="7" d148="7" d149="7" d150="7" d151="7" d152="7" d153="7" d154="7" d155="7" d156="7" d157="7" d158="7" d159="7" d160="7" d161="7" d162="7" d163="7" d164="7" d165="7" d166="7" d167="7" d168="7" d169="7" d170="7" d171="7" d172="7" d173="7" d174="7" d175="7" d176="7" d177="7" d178="7" d179="7" d180="7" d181="7" d182="7" d183="7" d184="7" d185="7" d186="7" d187="7" d188="7" d189="7" d190="7" d191="7" d192="7" d193="7" d194="7" d195="7" d196="7" d197="7" d198="7" d199="7" d200="7" d201="7" d202="7" d203="7" d204="7" d205="7" d206="7" d207="7" d208="7" d209="7" d210="7" d211="7" d212="7" d213="7" d214="7" d215="7" d216="7" d217="7" d218="7" d219="7" d220="7" d221="7" d222="7" d223="7" d224="7" d225="7" d226="7" d227="7" d228="7" d229="7" d230="7" d231="7" d232="7" d233="7" d234="7" d235="7" d236="7" d237="7" d238="7" d239="7" d240="7" d241="7" d242="7" d243="7" d244="7" d245="7" d246="7" d247="7" d248="7" d249="7" d250="7" d251="7" d252="7" d253="7" d254="7" d255="7" d256="7" d257="7" d258="7" d259="7" d260="7" d261="7" d262="7" d263="7" d264="7" d265="7" d266="7" d267="7" d268="7" d269="7" d270="7" d271="7" d272="7" d273="7" d274="7" d275="7" d276="7" d277="7" d278="7" d279="7" d280="7" d281="7" d282="7" d283="7" d284="7" d285="7" d286="7" d287="7" d288="7" d289="7" d290="7" d291="7" d292="7" d293="7" d294="7" d295="7" d296="7" d297="7" d298="7" d299="7" d300="7" d301="7" d302="7" d303="7" d304="7" d305="7" d306="7" d307="7" d308="7" d309="7" d310="7" d311="7" d312="7" d313="7" d314="7" d315="7" d316="7" d317="7" d318="7" d319="7" d320="7" d321="7" d322="7" d323="7" d324="7" d325="7" d326="7" d327="7" d328="7" d329="7" d330="7" d331="7" d332="7" d333="7" d334="7" d335="7" d336="7" d337="7" d338="7" d339="7" d340="7" d341="7" d342="7" d343="7" d344="7" d345="7" d346="7" d347="7" d348="7" d349="7" d350="7" d351="7" d352="7" d353="7" d354="7" d355="7" d356="7" d357="7" d358="7" d359="7" d360="7" d361="7" d362="7" d363="7" d364="7" d365="7"/>\n' +
        '<OccupancyYearProfile id="8" d1="8" d2="8" d3="8" d4="8" d5="8" d6="8" d7="8" d8="8" d9="8" d10="8" d11="8" d12="8" d13="8" d14="8" d15="8" d16="8" d17="8" d18="8" d19="8" d20="8" d21="8" d22="8" d23="8" d24="8" d25="8" d26="8" d27="8" d28="8" d29="8" d30="8" d31="8" d32="8" d33="8" d34="8" d35="8" d36="8" d37="8" d38="8" d39="8" d40="8" d41="8" d42="8" d43="8" d44="8" d45="8" d46="8" d47="8" d48="8" d49="8" d50="8" d51="8" d52="8" d53="8" d54="8" d55="8" d56="8" d57="8" d58="8" d59="8" d60="8" d61="8" d62="8" d63="8" d64="8" d65="8" d66="8" d67="8" d68="8" d69="8" d70="8" d71="8" d72="8" d73="8" d74="8" d75="8" d76="8" d77="8" d78="8" d79="8" d80="8" d81="8" d82="8" d83="8" d84="8" d85="8" d86="8" d87="8" d88="8" d89="8" d90="8" d91="8" d92="8" d93="8" d94="8" d95="8" d96="8" d97="8" d98="8" d99="8" d100="8" d101="8" d102="8" d103="8" d104="8" d105="8" d106="8" d107="8" d108="8" d109="8" d110="8" d111="8" d112="8" d113="8" d114="8" d115="8" d116="8" d117="8" d118="8" d119="8" d120="8" d121="8" d122="8" d123="8" d124="8" d125="8" d126="8" d127="8" d128="8" d129="8" d130="8" d131="8" d132="8" d133="8" d134="8" d135="8" d136="8" d137="8" d138="8" d139="8" d140="8" d141="8" d142="8" d143="8" d144="8" d145="8" d146="8" d147="8" d148="8" d149="8" d150="8" d151="8" d152="8" d153="8" d154="8" d155="8" d156="8" d157="8" d158="8" d159="8" d160="8" d161="8" d162="8" d163="8" d164="8" d165="8" d166="8" d167="8" d168="8" d169="8" d170="8" d171="8" d172="8" d173="8" d174="8" d175="8" d176="8" d177="8" d178="8" d179="8" d180="8" d181="8" d182="8" d183="8" d184="8" d185="8" d186="8" d187="8" d188="8" d189="8" d190="8" d191="8" d192="8" d193="8" d194="8" d195="8" d196="8" d197="8" d198="8" d199="8" d200="8" d201="8" d202="8" d203="8" d204="8" d205="8" d206="8" d207="8" d208="8" d209="8" d210="8" d211="8" d212="8" d213="8" d214="8" d215="8" d216="8" d217="8" d218="8" d219="8" d220="8" d221="8" d222="8" d223="8" d224="8" d225="8" d226="8" d227="8" d228="8" d229="8" d230="8" d231="8" d232="8" d233="8" d234="8" d235="8" d236="8" d237="8" d238="8" d239="8" d240="8" d241="8" d242="8" d243="8" d244="8" d245="8" d246="8" d247="8" d248="8" d249="8" d250="8" d251="8" d252="8" d253="8" d254="8" d255="8" d256="8" d257="8" d258="8" d259="8" d260="8" d261="8" d262="8" d263="8" d264="8" d265="8" d266="8" d267="8" d268="8" d269="8" d270="8" d271="8" d272="8" d273="8" d274="8" d275="8" d276="8" d277="8" d278="8" d279="8" d280="8" d281="8" d282="8" d283="8" d284="8" d285="8" d286="8" d287="8" d288="8" d289="8" d290="8" d291="8" d292="8" d293="8" d294="8" d295="8" d296="8" d297="8" d298="8" d299="8" d300="8" d301="8" d302="8" d303="8" d304="8" d305="8" d306="8" d307="8" d308="8" d309="8" d310="8" d311="8" d312="8" d313="8" d314="8" d315="8" d316="8" d317="8" d318="8" d319="8" d320="8" d321="8" d322="8" d323="8" d324="8" d325="8" d326="8" d327="8" d328="8" d329="8" d330="8" d331="8" d332="8" d333="8" d334="8" d335="8" d336="8" d337="8" d338="8" d339="8" d340="8" d341="8" d342="8" d343="8" d344="8" d345="8" d346="8" d347="8" d348="8" d349="8" d350="8" d351="8" d352="8" d353="8" d354="8" d355="8" d356="8" d357="8" d358="8" d359="8" d360="8" d361="8" d362="8" d363="8" d364="8" d365="8"/>\n' +
        '<OccupancyYearProfile id="9" d1="9" d2="9" d3="9" d4="9" d5="9" d6="9" d7="9" d8="9" d9="9" d10="9" d11="9" d12="9" d13="9" d14="9" d15="9" d16="9" d17="9" d18="9" d19="9" d20="9" d21="9" d22="9" d23="9" d24="9" d25="9" d26="9" d27="9" d28="9" d29="9" d30="9" d31="9" d32="9" d33="9" d34="9" d35="9" d36="9" d37="9" d38="9" d39="9" d40="9" d41="9" d42="9" d43="9" d44="9" d45="9" d46="9" d47="9" d48="9" d49="9" d50="9" d51="9" d52="9" d53="9" d54="9" d55="9" d56="9" d57="9" d58="9" d59="9" d60="9" d61="9" d62="9" d63="9" d64="9" d65="9" d66="9" d67="9" d68="9" d69="9" d70="9" d71="9" d72="9" d73="9" d74="9" d75="9" d76="9" d77="9" d78="9" d79="9" d80="9" d81="9" d82="9" d83="9" d84="9" d85="9" d86="9" d87="9" d88="9" d89="9" d90="9" d91="9" d92="9" d93="9" d94="9" d95="9" d96="9" d97="9" d98="9" d99="9" d100="9" d101="9" d102="9" d103="9" d104="9" d105="9" d106="9" d107="9" d108="9" d109="9" d110="9" d111="9" d112="9" d113="9" d114="9" d115="9" d116="9" d117="9" d118="9" d119="9" d120="9" d121="9" d122="9" d123="9" d124="9" d125="9" d126="9" d127="9" d128="9" d129="9" d130="9" d131="9" d132="9" d133="9" d134="9" d135="9" d136="9" d137="9" d138="9" d139="9" d140="9" d141="9" d142="9" d143="9" d144="9" d145="9" d146="9" d147="9" d148="9" d149="9" d150="9" d151="9" d152="9" d153="9" d154="9" d155="9" d156="9" d157="9" d158="9" d159="9" d160="9" d161="9" d162="9" d163="9" d164="9" d165="9" d166="9" d167="9" d168="9" d169="9" d170="9" d171="9" d172="9" d173="9" d174="9" d175="9" d176="9" d177="9" d178="9" d179="9" d180="9" d181="9" d182="9" d183="9" d184="9" d185="9" d186="9" d187="9" d188="9" d189="9" d190="9" d191="9" d192="9" d193="9" d194="9" d195="9" d196="9" d197="9" d198="9" d199="9" d200="9" d201="9" d202="9" d203="9" d204="9" d205="9" d206="9" d207="9" d208="9" d209="9" d210="9" d211="9" d212="9" d213="9" d214="9" d215="9" d216="9" d217="9" d218="9" d219="9" d220="9" d221="9" d222="9" d223="9" d224="9" d225="9" d226="9" d227="9" d228="9" d229="9" d230="9" d231="9" d232="9" d233="9" d234="9" d235="9" d236="9" d237="9" d238="9" d239="9" d240="9" d241="9" d242="9" d243="9" d244="9" d245="9" d246="9" d247="9" d248="9" d249="9" d250="9" d251="9" d252="9" d253="9" d254="9" d255="9" d256="9" d257="9" d258="9" d259="9" d260="9" d261="9" d262="9" d263="9" d264="9" d265="9" d266="9" d267="9" d268="9" d269="9" d270="9" d271="9" d272="9" d273="9" d274="9" d275="9" d276="9" d277="9" d278="9" d279="9" d280="9" d281="9" d282="9" d283="9" d284="9" d285="9" d286="9" d287="9" d288="9" d289="9" d290="9" d291="9" d292="9" d293="9" d294="9" d295="9" d296="9" d297="9" d298="9" d299="9" d300="9" d301="9" d302="9" d303="9" d304="9" d305="9" d306="9" d307="9" d308="9" d309="9" d310="9" d311="9" d312="9" d313="9" d314="9" d315="9" d316="9" d317="9" d318="9" d319="9" d320="9" d321="9" d322="9" d323="9" d324="9" d325="9" d326="9" d327="9" d328="9" d329="9" d330="9" d331="9" d332="9" d333="9" d334="9" d335="9" d336="9" d337="9" d338="9" d339="9" d340="9" d341="9" d342="9" d343="9" d344="9" d345="9" d346="9" d347="9" d348="9" d349="9" d350="9" d351="9" d352="9" d353="9" d354="9" d355="9" d356="9" d357="9" d358="9" d359="9" d360="9" d361="9" d362="9" d363="9" d364="9" d365="9"/>\n' +
        '<OccupancyYearProfile id="10" d1="10" d2="10" d3="10" d4="10" d5="10" d6="10" d7="10" d8="10" d9="10" d10="10" d11="10" d12="10" d13="10" d14="10" d15="10" d16="10" d17="10" d18="10" d19="10" d20="10" d21="10" d22="10" d23="10" d24="10" d25="10" d26="10" d27="10" d28="10" d29="10" d30="10" d31="10" d32="10" d33="10" d34="10" d35="10" d36="10" d37="10" d38="10" d39="10" d40="10" d41="10" d42="10" d43="10" d44="10" d45="10" d46="10" d47="10" d48="10" d49="10" d50="10" d51="10" d52="10" d53="10" d54="10" d55="10" d56="10" d57="10" d58="10" d59="10" d60="10" d61="10" d62="10" d63="10" d64="10" d65="10" d66="10" d67="10" d68="10" d69="10" d70="10" d71="10" d72="10" d73="10" d74="10" d75="10" d76="10" d77="10" d78="10" d79="10" d80="10" d81="10" d82="10" d83="10" d84="10" d85="10" d86="10" d87="10" d88="10" d89="10" d90="10" d91="10" d92="10" d93="10" d94="10" d95="10" d96="10" d97="10" d98="10" d99="10" d100="10" d101="10" d102="10" d103="10" d104="10" d105="10" d106="10" d107="10" d108="10" d109="10" d110="10" d111="10" d112="10" d113="10" d114="10" d115="10" d116="10" d117="10" d118="10" d119="10" d120="10" d121="10" d122="10" d123="10" d124="10" d125="10" d126="10" d127="10" d128="10" d129="10" d130="10" d131="10" d132="10" d133="10" d134="10" d135="10" d136="10" d137="10" d138="10" d139="10" d140="10" d141="10" d142="10" d143="10" d144="10" d145="10" d146="10" d147="10" d148="10" d149="10" d150="10" d151="10" d152="10" d153="10" d154="10" d155="10" d156="10" d157="10" d158="10" d159="10" d160="10" d161="10" d162="10" d163="10" d164="10" d165="10" d166="10" d167="10" d168="10" d169="10" d170="10" d171="10" d172="10" d173="10" d174="10" d175="10" d176="10" d177="10" d178="10" d179="10" d180="10" d181="10" d182="10" d183="10" d184="10" d185="10" d186="10" d187="10" d188="10" d189="10" d190="10" d191="10" d192="10" d193="10" d194="10" d195="10" d196="10" d197="10" d198="10" d199="10" d200="10" d201="10" d202="10" d203="10" d204="10" d205="10" d206="10" d207="10" d208="10" d209="10" d210="10" d211="10" d212="10" d213="10" d214="10" d215="10" d216="10" d217="10" d218="10" d219="10" d220="10" d221="10" d222="10" d223="10" d224="10" d225="10" d226="10" d227="10" d228="10" d229="10" d230="10" d231="10" d232="10" d233="10" d234="10" d235="10" d236="10" d237="10" d238="10" d239="10" d240="10" d241="10" d242="10" d243="10" d244="10" d245="10" d246="10" d247="10" d248="10" d249="10" d250="10" d251="10" d252="10" d253="10" d254="10" d255="10" d256="10" d257="10" d258="10" d259="10" d260="10" d261="10" d262="10" d263="10" d264="10" d265="10" d266="10" d267="10" d268="10" d269="10" d270="10" d271="10" d272="10" d273="10" d274="10" d275="10" d276="10" d277="10" d278="10" d279="10" d280="10" d281="10" d282="10" d283="10" d284="10" d285="10" d286="10" d287="10" d288="10" d289="10" d290="10" d291="10" d292="10" d293="10" d294="10" d295="10" d296="10" d297="10" d298="10" d299="10" d300="10" d301="10" d302="10" d303="10" d304="10" d305="10" d306="10" d307="10" d308="10" d309="10" d310="10" d311="10" d312="10" d313="10" d314="10" d315="10" d316="10" d317="10" d318="10" d319="10" d320="10" d321="10" d322="10" d323="10" d324="10" d325="10" d326="10" d327="10" d328="10" d329="10" d330="10" d331="10" d332="10" d333="10" d334="10" d335="10" d336="10" d337="10" d338="10" d339="10" d340="10" d341="10" d342="10" d343="10" d344="10" d345="10" d346="10" d347="10" d348="10" d349="10" d350="10" d351="10" d352="10" d353="10" d354="10" d355="10" d356="10" d357="10" d358="10" d359="10" d360="10" d361="10" d362="10" d363="10" d364="10" d365="10"/>',
        file=open(xml_name, "a"))

# Define devices and their activities #####################################
print('<DeviceType id="1" name="Residential">\n' +
          '<Device  name="Residential device"  avgPower="190"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.5" p8="1.0" p9="0.5" p10="0.5" p11="0.5" p12="1.0" p13="1.0" p14="0.5" p15="0.5" p16="0.5" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="0.5" p22="0.5" p23="0.5" p24="1.0"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="2" name="Office">\n' +
          '<Device  name="Office device"  avgPower="130"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.6" p11="0.8" p12="0.8" p13="0.4" p14="0.6" p15="0.8" p16="0.8" p17="0.4" p18="0.2" p19="0.001" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="3" name="Sport center">\n' +
          '<Device  name="Sport center device"  avgPower="40"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.6" p10="1" p11="1" p12="0.2" p13="0.2" p14="1" p15="1" p16="0.6" p17="0.6" p18="0.6" p19="0.6" p20="1" p21="1" p22="0.6" p23="0.2" p24="0.001"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="4" name="Commercial">\n' +
          '<Device  name="Commercial device"  avgPower="20"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="1" p9="1" p10="1" p11="1" p12="1" p13="1" p14="1" p15="1" p16="1" p17="1" p18="1" p19="1" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="5" name="Restaurant">\n' +
          '<Device  name="Restaurant device"  avgPower="4"  convectiveFraction="0.8" radiativeFraction="0.1" p1="100" p2="100" p3="100" p4="100" p5="100" p6="100" p7="100" p8="100" p9="2" p10="2" p11="2" p12="2" p13="1.25" p14="2.5" p15="100" p16="100" p17="100" p18="100" p19="2" p20="2" p21="1" p22="1.25" p23="4" p24="2"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="6" name="Hotel">\n' +
          '<Device  name="Hotel device"  avgPower="68"  convectiveFraction="0.8" radiativeFraction="0.1" p1="1" p2="0.4" p3="0.4" p4="0.4" p5="0.4" p6="0.4" p7="0.4" p8="0.4" p9="0.4" p10="0.4" p11="0.4" p12="0.4" p13="0.4" p14="0.4" p15="0.4" p16="0.4" p17="0.4" p18="0.4" p19="0.4" p20="0.4" p21="0.4" p22="0.6" p23="0.8" p24="1"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="7" name="Hospital">\n' +
          '<Device  name="Hospital device"  avgPower="60"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.2" p2="0.2" p3="0.2" p4="0.2" p5="0.2" p6="0.2" p7="0.2" p8="0.2" p9="0.2" p10="0.2" p11="0.2" p12="0.2" p13="0.2" p14="0.2" p15="0.2" p16="0.2" p17="0.2" p18="0.2" p19="0.2" p20="0.8" p21="0.6" p22="0.4" p23="0.2" p24="0.2"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="8" name="Education">\n' +
          '<Device  name="Education device"  avgPower="40"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.001" p2="0.001" p3="0.001" p4="0.001" p5="0.001" p6="0.001" p7="0.001" p8="0.2" p9="0.4" p10="0.8" p11="0.8" p12="0.4" p13="0.2" p14="0.4" p15="0.8" p16="0.8" p17="0.2" p18="0.001" p19="0.001" p20="0.001" p21="0.001" p22="0.001" p23="0.001" p24="0.001"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="9" name="Industrial">\n' +
          '<Device  name="Industrial device"  avgPower="300"  convectiveFraction="0.8" radiativeFraction="0.1" p1="0.2" p2="0.2" p3="0.2" p4="0.2" p5="0.2" p6="0.5" p7="0.8" p8="1" p9="1" p10="0.8" p11="1" p12="0.5" p13="0.8" p14="1" p15="1" p16="0.8" p17="0.8" p18="0.8" p19="0.5" p20="0.5" p21="0.5" p22="0.5" p23="0.2" p24="0.2"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="10" name="Other">\n' +
          '<Device name="Other device"  avgPower="130"  convectiveFraction="0.8" radiativeFraction="0.1" p1="100" p2="100" p3="100" p4="100" p5="100" p6="100" p7="100" p8="1" p9="1" p10="1" p11="1" p12="1" p13="1" p14="1" p15="1" p16="1" p17="1" p18="1" p19="100" p20="100" p21="100" p22="100" p23="100" p24="100"/>\n' +
          '</DeviceType>\n' +
          '<DeviceType id="11" name="Othe">\n' +
          '<Device name="Othe device"  avgPower="0.1"  convectiveFraction="0.8" radiativeFraction="0.1" p1="100" p2="100" p3="100" p4="100" p5="100" p6="100" p7="100" p8="1" p9="1" p10="1" p11="1" p12="1" p13="1" p14="1" p15="1" p16="1" p17="1" p18="1" p19="100" p20="100" p21="100" p22="100" p23="100" p24="100"/>\n' +
          '</DeviceType>\n' +
          '<ActivityType id="1" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="1"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="2" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="2"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="3" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="3"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="4" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="4"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="5" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="5"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="6" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="6"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="7" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="7"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="8" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="8"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="9" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="9"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="10" name="Standard">\n' +
          '<Activity name="Standard" p1="1.0" p2="1.0" p3="1.0" p4="1.0" p5="1.0" p6="1.0" p7="1.0" p8="1.0" p9="1.0" p10="1.0" p11="1.0" p12="1.0" p13="1.0" p14="1.0" p15="1.0" p16="1.0" p17="1.0" p18="1.0" p19="1.0" p20="1.0" p21="1.0" p22="1.0" p23="1.0" p24="1.0" deviceType="10"/>\n' +
          '</ActivityType>\n' +
          '<ActivityType id="11" name="Standard">\n' +
          '<Activity name="Standard" p1="0.0" p2="0.0" p3="0.0" p4="0.0" p5="0.0" p6="0.0" p7="0.0" p8="0.0" p9="0.0" p10="0.0" p11="0.0" p12="0.0" p13="0.0" p14="0.0" p15="0.0" p16="0.0" p17="0.0" p18="0.0" p19="0.0" p20="0.0" p21="0.0" p22="0.0" p23="0.0" p24="0.0" deviceType="11"/>\n' +
          '</ActivityType>',
          file=open(xml_name, "a"))

# Define IEA tasks ########################################################
print(
        '<DHWDayProfile id="1" name=" IEA task 26 DHW profile 1" waterConsumption="50.0" p1="0.01" p2="0.01" p3="0.01" p4="0.01" p5="0.4" p6="0.07" p7="0.16" p8="0.12" p9="0.10" p10="0.09" p11="0.10" p12="0.07" p13="0.05" p14="0.07" p15="0.10" p16="0.10" p17="0.07" p18="0.06" p19="0.10" p20="0.09" p21="0.05" p22="0.04" p23="0.01" p24="0.01"/>\n' +
        '<DHWDayProfile id="2" name=" IEA task 26 DHW profile 2" waterConsumption="0.0" p1="0.0" p2="0.0" p3="0.0" p4="0.0" p5="0.0" p6="0.0" p7="0.0" p8="0.0" p9="0.0" p10="0.0" p11="0.0" p12="0.0" p13="0.0" p14="0.0" p15="0.0" p16="0.0" p17="0.0" p18="0.0" p19="0.0" p20="0.0" p21="0.0" p22="0.0" p23="0.0" p24="0.0"/>\n' +
        '<DHWYearProfile id="1" name="IEA task 26 Profile House 1" d1="1" d2="1" d3="1" d4="1" d5="1" d6="1" d7="1" d8="1" d9="1" d10="1" d11="1" d12="1" d13="1" d14="1" d15="1" d16="1" d17="1" d18="1" d19="1" d20="1" d21="1" d22="1" d23="1" d24="1" d25="1" d26="1" d27="1" d28="1" d29="1" d30="1" d31="1" d32="1" d33="1" d34="1" d35="1" d36="1" d37="1" d38="1" d39="1" d40="1" d41="1" d42="1" d43="1" d44="1" d45="1" d46="1" d47="1" d48="1" d49="1" d50="1" d51="1" d52="1" d53="1" d54="1" d55="1" d56="1" d57="1" d58="1" d59="1" d60="1" d61="1" d62="1" d63="1" d64="1" d65="1" d66="1" d67="1" d68="1" d69="1" d70="1" d71="1" d72="1" d73="1" d74="1" d75="1" d76="1" d77="1" d78="1" d79="1" d80="1" d81="1" d82="1" d83="1" d84="1" d85="1" d86="1" d87="1" d88="1" d89="1" d90="1" d91="1" d92="1" d93="1" d94="1" d95="1" d96="1" d97="1" d98="1" d99="1" d100="1" d101="1" d102="1" d103="1" d104="1" d105="1" d106="1" d107="1" d108="1" d109="1" d110="1" d111="1" d112="1" d113="1" d114="1" d115="1" d116="1" d117="1" d118="1" d119="1" d120="1" d121="1" d122="1" d123="1" d124="1" d125="1" d126="1" d127="1" d128="1" d129="1" d130="1" d131="1" d132="1" d133="1" d134="1" d135="1" d136="1" d137="1" d138="1" d139="1" d140="1" d141="1" d142="1" d143="1" d144="1" d145="1" d146="1" d147="1" d148="1" d149="1" d150="1" d151="1" d152="1" d153="1" d154="1" d155="1" d156="1" d157="1" d158="1" d159="1" d160="1" d161="1" d162="1" d163="1" d164="1" d165="1" d166="1" d167="1" d168="1" d169="1" d170="1" d171="1" d172="1" d173="1" d174="1" d175="1" d176="1" d177="1" d178="1" d179="1" d180="1" d181="1" d182="1" d183="1" d184="1" d185="1" d186="1" d187="1" d188="1" d189="1" d190="1" d191="1" d192="1" d193="1" d194="1" d195="1" d196="1" d197="1" d198="1" d199="1" d200="1" d201="1" d202="1" d203="1" d204="1" d205="1" d206="1" d207="1" d208="1" d209="1" d210="1" d211="1" d212="1" d213="1" d214="1" d215="1" d216="1" d217="1" d218="1" d219="1" d220="1" d221="1" d222="1" d223="1" d224="1" d225="1" d226="1" d227="1" d228="1" d229="1" d230="1" d231="1" d232="1" d233="1" d234="1" d235="1" d236="1" d237="1" d238="1" d239="1" d240="1" d241="1" d242="1" d243="1" d244="1" d245="1" d246="1" d247="1" d248="1" d249="1" d250="1" d251="1" d252="1" d253="1" d254="1" d255="1" d256="1" d257="1" d258="1" d259="1" d260="1" d261="1" d262="1" d263="1" d264="1" d265="1" d266="1" d267="1" d268="1" d269="1" d270="1" d271="1" d272="1" d273="1" d274="1" d275="1" d276="1" d277="1" d278="1" d279="1" d280="1" d281="1" d282="1" d283="1" d284="1" d285="1" d286="1" d287="1" d288="1" d289="1" d290="1" d291="1" d292="1" d293="1" d294="1" d295="1" d296="1" d297="1" d298="1" d299="1" d300="1" d301="1" d302="1" d303="1" d304="1" d305="1" d306="1" d307="1" d308="1" d309="1" d310="1" d311="1" d312="1" d313="1" d314="1" d315="1" d316="1" d317="1" d318="1" d319="1" d320="1" d321="1" d322="1" d323="1" d324="1" d325="1" d326="1" d327="1" d328="1" d329="1" d330="1" d331="1" d332="1" d333="1" d334="1" d335="1" d336="1" d337="1" d338="1" d339="1" d340="1" d341="1" d342="1" d343="1" d344="1" d345="1" d346="1" d347="1" d348="1" d349="1" d350="1" d351="1" d352="1" d353="1" d354="1" d355="1" d356="1" d357="1" d358="1" d359="1" d360="1" d361="1" d362="1" d363="1" d364="1" d365="1"/>\n' +
        '<DHWYearProfile id="2" name="IEA task 26 Profile House 2" d1="2" d2="2" d3="2" d4="2" d5="2" d6="2" d7="2" d8="2" d9="2" d10="2" d11="2" d12="2" d13="2" d14="2" d15="2" d16="2" d17="2" d18="2" d19="2" d20="2" d21="2" d22="2" d23="2" d24="2" d25="2" d26="2" d27="2" d28="2" d29="2" d30="2" d31="2" d32="2" d33="2" d34="2" d35="2" d36="2" d37="2" d38="2" d39="2" d40="2" d41="2" d42="2" d43="2" d44="2" d45="2" d46="2" d47="2" d48="2" d49="2" d50="2" d51="2" d52="2" d53="2" d54="2" d55="2" d56="2" d57="2" d58="2" d59="2" d60="2" d61="2" d62="2" d63="2" d64="2" d65="2" d66="2" d67="2" d68="2" d69="2" d70="2" d71="2" d72="2" d73="2" d74="2" d75="2" d76="2" d77="2" d78="2" d79="2" d80="2" d81="2" d82="2" d83="2" d84="2" d85="2" d86="2" d87="2" d88="2" d89="2" d90="2" d91="2" d92="2" d93="2" d94="2" d95="2" d96="2" d97="2" d98="2" d99="2" d100="2" d101="2" d102="2" d103="2" d104="2" d105="2" d106="2" d107="2" d108="2" d109="2" d110="2" d111="2" d112="2" d113="2" d114="2" d115="2" d116="2" d117="2" d118="2" d119="2" d120="2" d121="2" d122="2" d123="2" d124="2" d125="2" d126="2" d127="2" d128="2" d129="2" d130="2" d131="2" d132="2" d133="2" d134="2" d135="2" d136="2" d137="2" d138="2" d139="2" d140="2" d141="2" d142="2" d143="2" d144="2" d145="2" d146="2" d147="2" d148="2" d149="2" d150="2" d151="2" d152="2" d153="2" d154="2" d155="2" d156="2" d157="2" d158="2" d159="2" d160="2" d161="2" d162="2" d163="2" d164="2" d165="2" d166="2" d167="2" d168="2" d169="2" d170="2" d171="2" d172="2" d173="2" d174="2" d175="2" d176="2" d177="2" d178="2" d179="2" d180="2" d181="2" d182="2" d183="2" d184="2" d185="2" d186="2" d187="2" d188="2" d189="2" d190="2" d191="2" d192="2" d193="2" d194="2" d195="2" d196="2" d197="2" d198="2" d199="2" d200="2" d201="2" d202="2" d203="2" d204="2" d205="2" d206="2" d207="2" d208="2" d209="2" d210="2" d211="2" d212="2" d213="2" d214="2" d215="2" d216="2" d217="2" d218="2" d219="2" d220="2" d221="2" d222="2" d223="2" d224="2" d225="2" d226="2" d227="2" d228="2" d229="2" d230="2" d231="2" d232="2" d233="2" d234="2" d235="2" d236="2" d237="2" d238="2" d239="2" d240="2" d241="2" d242="2" d243="2" d244="2" d245="2" d246="2" d247="2" d248="2" d249="2" d250="2" d251="2" d252="2" d253="2" d254="2" d255="2" d256="2" d257="2" d258="2" d259="2" d260="2" d261="2" d262="2" d263="2" d264="2" d265="2" d266="2" d267="2" d268="2" d269="2" d270="2" d271="2" d272="2" d273="2" d274="2" d275="2" d276="2" d277="2" d278="2" d279="2" d280="2" d281="2" d282="2" d283="2" d284="2" d285="2" d286="2" d287="2" d288="2" d289="2" d290="2" d291="2" d292="2" d293="2" d294="2" d295="2" d296="2" d297="2" d298="2" d299="2" d300="2" d301="2" d302="2" d303="2" d304="2" d305="2" d306="2" d307="2" d308="2" d309="2" d310="2" d311="2" d312="2" d313="2" d314="2" d315="2" d316="2" d317="2" d318="2" d319="2" d320="2" d321="2" d322="2" d323="2" d324="2" d325="2" d326="2" d327="2" d328="2" d329="2" d330="2" d331="2" d332="2" d333="2" d334="2" d335="2" d336="2" d337="2" d338="2" d339="2" d340="2" d341="2" d342="2" d343="2" d344="2" d345="2" d346="2" d347="2" d348="2" d349="2" d350="2" d351="2" d352="2" d353="2" d354="2" d355="2" d356="2" d357="2" d358="2" d359="2" d360="2" d361="2" d362="2" d363="2" d364="2" d365="2"/>',
        file=open(xml_name, "a"))

# Define devices yearly profiles according to occupancy_type ##############
print(
        '<OccupancyYearProfile id="1" d1="1" d2="1" d3="1" d4="1" d5="1" d6="1" d7="1" d8="1" d9="1" d10="1" d11="1" d12="1" d13="1" d14="1" d15="1" d16="1" d17="1" d18="1" d19="1" d20="1" d21="1" d22="1" d23="1" d24="1" d25="1" d26="1" d27="1" d28="1" d29="1" d30="1" d31="1" d32="1" d33="1" d34="1" d35="1" d36="1" d37="1" d38="1" d39="1" d40="1" d41="1" d42="1" d43="1" d44="1" d45="1" d46="1" d47="1" d48="1" d49="1" d50="1" d51="1" d52="1" d53="1" d54="1" d55="1" d56="1" d57="1" d58="1" d59="1" d60="1" d61="1" d62="1" d63="1" d64="1" d65="1" d66="1" d67="1" d68="1" d69="1" d70="1" d71="1" d72="1" d73="1" d74="1" d75="1" d76="1" d77="1" d78="1" d79="1" d80="1" d81="1" d82="1" d83="1" d84="1" d85="1" d86="1" d87="1" d88="1" d89="1" d90="1" d91="1" d92="1" d93="1" d94="1" d95="1" d96="1" d97="1" d98="1" d99="1" d100="1" d101="1" d102="1" d103="1" d104="1" d105="1" d106="1" d107="1" d108="1" d109="1" d110="1" d111="1" d112="1" d113="1" d114="1" d115="1" d116="1" d117="1" d118="1" d119="1" d120="1" d121="1" d122="1" d123="1" d124="1" d125="1" d126="1" d127="1" d128="1" d129="1" d130="1" d131="1" d132="1" d133="1" d134="1" d135="1" d136="1" d137="1" d138="1" d139="1" d140="1" d141="1" d142="1" d143="1" d144="1" d145="1" d146="1" d147="1" d148="1" d149="1" d150="1" d151="1" d152="1" d153="1" d154="1" d155="1" d156="1" d157="1" d158="1" d159="1" d160="1" d161="1" d162="1" d163="1" d164="1" d165="1" d166="1" d167="1" d168="1" d169="1" d170="1" d171="1" d172="1" d173="1" d174="1" d175="1" d176="1" d177="1" d178="1" d179="1" d180="1" d181="1" d182="1" d183="1" d184="1" d185="1" d186="1" d187="1" d188="1" d189="1" d190="1" d191="1" d192="1" d193="1" d194="1" d195="1" d196="1" d197="1" d198="1" d199="1" d200="1" d201="1" d202="1" d203="1" d204="1" d205="1" d206="1" d207="1" d208="1" d209="1" d210="1" d211="1" d212="1" d213="1" d214="1" d215="1" d216="1" d217="1" d218="1" d219="1" d220="1" d221="1" d222="1" d223="1" d224="1" d225="1" d226="1" d227="1" d228="1" d229="1" d230="1" d231="1" d232="1" d233="1" d234="1" d235="1" d236="1" d237="1" d238="1" d239="1" d240="1" d241="1" d242="1" d243="1" d244="1" d245="1" d246="1" d247="1" d248="1" d249="1" d250="1" d251="1" d252="1" d253="1" d254="1" d255="1" d256="1" d257="1" d258="1" d259="1" d260="1" d261="1" d262="1" d263="1" d264="1" d265="1" d266="1" d267="1" d268="1" d269="1" d270="1" d271="1" d272="1" d273="1" d274="1" d275="1" d276="1" d277="1" d278="1" d279="1" d280="1" d281="1" d282="1" d283="1" d284="1" d285="1" d286="1" d287="1" d288="1" d289="1" d290="1" d291="1" d292="1" d293="1" d294="1" d295="1" d296="1" d297="1" d298="1" d299="1" d300="1" d301="1" d302="1" d303="1" d304="1" d305="1" d306="1" d307="1" d308="1" d309="1" d310="1" d311="1" d312="1" d313="1" d314="1" d315="1" d316="1" d317="1" d318="1" d319="1" d320="1" d321="1" d322="1" d323="1" d324="1" d325="1" d326="1" d327="1" d328="1" d329="1" d330="1" d331="1" d332="1" d333="1" d334="1" d335="1" d336="1" d337="1" d338="1" d339="1" d340="1" d341="1" d342="1" d343="1" d344="1" d345="1" d346="1" d347="1" d348="1" d349="1" d350="1" d351="1" d352="1" d353="1" d354="1" d355="1" d356="1" d357="1" d358="1" d359="1" d360="1" d361="1" d362="1" d363="1" d364="1" d365="1"/>\n' +
        '<OccupancyYearProfile id="2" d1="2" d2="2" d3="2" d4="2" d5="2" d6="0" d7="0" d8="2" d9="2" d10="2" d11="2" d12="2" d13="0" d14="0" d15="2" d16="2" d17="2" d18="2" d19="2" d20="0" d21="0" d22="2" d23="2" d24="2" d25="2" d26="2" d27="0" d28="0" d29="2" d30="2" d31="2" d32="2" d33="2" d34="0" d35="0" d36="2" d37="2" d38="2" d39="2" d40="2" d41="0" d42="0" d43="2" d44="2" d45="2" d46="2" d47="2" d48="0" d49="0" d50="2" d51="2" d52="2" d53="2" d54="2" d55="0" d56="0" d57="2" d58="2" d59="2" d60="2" d61="2" d62="0" d63="0" d64="2" d65="2" d66="2" d67="2" d68="2" d69="0" d70="0" d71="2" d72="2" d73="2" d74="2" d75="2" d76="0" d77="0" d78="2" d79="2" d80="2" d81="2" d82="2" d83="0" d84="0" d85="2" d86="2" d87="2" d88="2" d89="2" d90="0" d91="0" d92="2" d93="2" d94="2" d95="2" d96="2" d97="0" d98="0" d99="2" d100="2" d101="2" d102="2" d103="2" d104="0" d105="0" d106="2" d107="2" d108="2" d109="2" d110="2" d111="0" d112="0" d113="2" d114="2" d115="2" d116="2" d117="2" d118="0" d119="0" d120="2" d121="2" d122="2" d123="2" d124="2" d125="0" d126="0" d127="2" d128="2" d129="2" d130="2" d131="2" d132="0" d133="0" d134="2" d135="2" d136="2" d137="2" d138="2" d139="0" d140="0" d141="2" d142="2" d143="2" d144="2" d145="2" d146="0" d147="0" d148="2" d149="2" d150="2" d151="2" d152="2" d153="0" d154="0" d155="2" d156="2" d157="2" d158="2" d159="2" d160="0" d161="0" d162="2" d163="2" d164="2" d165="2" d166="2" d167="0" d168="0" d169="2" d170="2" d171="2" d172="2" d173="2" d174="0" d175="0" d176="2" d177="2" d178="2" d179="2" d180="2" d181="0" d182="0" d183="2" d184="2" d185="2" d186="2" d187="2" d188="0" d189="0" d190="2" d191="2" d192="2" d193="2" d194="2" d195="0" d196="0" d197="2" d198="2" d199="2" d200="2" d201="2" d202="0" d203="0" d204="2" d205="2" d206="2" d207="2" d208="2" d209="0" d210="0" d211="2" d212="2" d213="2" d214="2" d215="2" d216="0" d217="0" d218="2" d219="2" d220="2" d221="2" d222="2" d223="0" d224="0" d225="2" d226="2" d227="2" d228="2" d229="2" d230="0" d231="0" d232="2" d233="2" d234="2" d235="2" d236="2" d237="0" d238="0" d239="2" d240="2" d241="2" d242="2" d243="2" d244="0" d245="0" d246="2" d247="2" d248="2" d249="2" d250="2" d251="0" d252="0" d253="2" d254="2" d255="2" d256="2" d257="2" d258="0" d259="0" d260="2" d261="2" d262="2" d263="2" d264="2" d265="2" d266="0" d267="0" d268="2" d269="2" d270="2" d271="2" d272="2" d273="0" d274="0" d275="2" d276="2" d277="2" d278="2" d279="2" d280="0" d281="0" d282="2" d283="2" d284="2" d285="2" d286="2" d287="0" d288="0" d289="2" d290="2" d291="2" d292="2" d293="2" d294="0" d295="0" d296="2" d297="2" d298="2" d299="2" d300="2" d301="0" d302="0" d303="2" d304="2" d305="2" d306="2" d307="2" d308="0" d309="0" d310="2" d311="2" d312="2" d313="2" d314="2" d315="0" d316="0" d317="2" d318="2" d319="2" d320="2" d321="2" d322="0" d323="0" d324="2" d325="2" d326="2" d327="2" d328="2" d329="0" d330="0" d331="2" d332="2" d333="2" d334="2" d335="2" d336="0" d337="0" d338="2" d339="2" d340="2" d341="2" d342="2" d343="0" d344="0" d345="2" d346="2" d347="2" d348="2" d349="2" d350="0" d351="0" d352="2" d353="2" d354="2" d355="2" d356="2" d357="0" d358="0" d359="2" d360="2" d361="2" d362="2" d363="2" d364="0" d365="0"/>\n' +
        '<OccupancyYearProfile id="3" d1="3" d2="3" d3="3" d4="3" d5="3" d6="3" d7="3" d8="3" d9="3" d10="3" d11="3" d12="3" d13="3" d14="3" d15="3" d16="3" d17="3" d18="3" d19="3" d20="3" d21="3" d22="3" d23="3" d24="3" d25="3" d26="3" d27="3" d28="3" d29="3" d30="3" d31="3" d32="3" d33="3" d34="3" d35="3" d36="3" d37="3" d38="3" d39="3" d40="3" d41="3" d42="3" d43="3" d44="3" d45="3" d46="3" d47="3" d48="3" d49="3" d50="3" d51="3" d52="3" d53="3" d54="3" d55="3" d56="3" d57="3" d58="3" d59="3" d60="3" d61="3" d62="3" d63="3" d64="3" d65="3" d66="3" d67="3" d68="3" d69="3" d70="3" d71="3" d72="3" d73="3" d74="3" d75="3" d76="3" d77="3" d78="3" d79="3" d80="3" d81="3" d82="3" d83="3" d84="3" d85="3" d86="3" d87="3" d88="3" d89="3" d90="3" d91="3" d92="3" d93="3" d94="3" d95="3" d96="3" d97="3" d98="3" d99="3" d100="3" d101="3" d102="3" d103="3" d104="3" d105="3" d106="3" d107="3" d108="3" d109="3" d110="3" d111="3" d112="3" d113="3" d114="3" d115="3" d116="3" d117="3" d118="3" d119="3" d120="3" d121="3" d122="3" d123="3" d124="3" d125="3" d126="3" d127="3" d128="3" d129="3" d130="3" d131="3" d132="3" d133="3" d134="3" d135="3" d136="3" d137="3" d138="3" d139="3" d140="3" d141="3" d142="3" d143="3" d144="3" d145="3" d146="3" d147="3" d148="3" d149="3" d150="3" d151="3" d152="3" d153="3" d154="3" d155="3" d156="3" d157="3" d158="3" d159="3" d160="3" d161="3" d162="3" d163="3" d164="3" d165="3" d166="3" d167="3" d168="3" d169="3" d170="3" d171="3" d172="3" d173="3" d174="3" d175="3" d176="3" d177="3" d178="3" d179="3" d180="3" d181="3" d182="3" d183="3" d184="3" d185="3" d186="3" d187="3" d188="3" d189="3" d190="3" d191="3" d192="3" d193="3" d194="3" d195="3" d196="3" d197="3" d198="3" d199="3" d200="3" d201="3" d202="3" d203="3" d204="3" d205="3" d206="3" d207="3" d208="3" d209="3" d210="3" d211="3" d212="3" d213="3" d214="3" d215="3" d216="3" d217="3" d218="3" d219="3" d220="3" d221="3" d222="3" d223="3" d224="3" d225="3" d226="3" d227="3" d228="3" d229="3" d230="3" d231="3" d232="3" d233="3" d234="3" d235="3" d236="3" d237="3" d238="3" d239="3" d240="3" d241="3" d242="3" d243="3" d244="3" d245="3" d246="3" d247="3" d248="3" d249="3" d250="3" d251="3" d252="3" d253="3" d254="3" d255="3" d256="3" d257="3" d258="3" d259="3" d260="3" d261="3" d262="3" d263="3" d264="3" d265="3" d266="3" d267="3" d268="3" d269="3" d270="3" d271="3" d272="3" d273="3" d274="3" d275="3" d276="3" d277="3" d278="3" d279="3" d280="3" d281="3" d282="3" d283="3" d284="3" d285="3" d286="3" d287="3" d288="3" d289="3" d290="3" d291="3" d292="3" d293="3" d294="3" d295="3" d296="3" d297="3" d298="3" d299="3" d300="3" d301="3" d302="3" d303="3" d304="3" d305="3" d306="3" d307="3" d308="3" d309="3" d310="3" d311="3" d312="3" d313="3" d314="3" d315="3" d316="3" d317="3" d318="3" d319="3" d320="3" d321="3" d322="3" d323="3" d324="3" d325="3" d326="3" d327="3" d328="3" d329="3" d330="3" d331="3" d332="3" d333="3" d334="3" d335="3" d336="3" d337="3" d338="3" d339="3" d340="3" d341="3" d342="3" d343="3" d344="3" d345="3" d346="3" d347="3" d348="3" d349="3" d350="3" d351="3" d352="3" d353="3" d354="3" d355="3" d356="3" d357="3" d358="3" d359="3" d360="3" d361="3" d362="3" d363="3" d364="3" d365="3"/>\n' +
        '<OccupancyYearProfile id="4" d1="4" d2="4" d3="4" d4="4" d5="4" d6="4" d7="4" d8="4" d9="4" d10="4" d11="4" d12="4" d13="4" d14="4" d15="4" d16="4" d17="4" d18="4" d19="4" d20="4" d21="4" d22="4" d23="4" d24="4" d25="4" d26="4" d27="4" d28="4" d29="4" d30="4" d31="4" d32="4" d33="4" d34="4" d35="4" d36="4" d37="4" d38="4" d39="4" d40="4" d41="4" d42="4" d43="4" d44="4" d45="4" d46="4" d47="4" d48="4" d49="4" d50="4" d51="4" d52="4" d53="4" d54="4" d55="4" d56="4" d57="4" d58="4" d59="4" d60="4" d61="4" d62="4" d63="4" d64="4" d65="4" d66="4" d67="4" d68="4" d69="4" d70="4" d71="4" d72="4" d73="4" d74="4" d75="4" d76="4" d77="4" d78="4" d79="4" d80="4" d81="4" d82="4" d83="4" d84="4" d85="4" d86="4" d87="4" d88="4" d89="4" d90="4" d91="4" d92="4" d93="4" d94="4" d95="4" d96="4" d97="4" d98="4" d99="4" d100="4" d101="4" d102="4" d103="4" d104="4" d105="4" d106="4" d107="4" d108="4" d109="4" d110="4" d111="4" d112="4" d113="4" d114="4" d115="4" d116="4" d117="4" d118="4" d119="4" d120="4" d121="4" d122="4" d123="4" d124="4" d125="4" d126="4" d127="4" d128="4" d129="4" d130="4" d131="4" d132="4" d133="4" d134="4" d135="4" d136="4" d137="4" d138="4" d139="4" d140="4" d141="4" d142="4" d143="4" d144="4" d145="4" d146="4" d147="4" d148="4" d149="4" d150="4" d151="4" d152="4" d153="4" d154="4" d155="4" d156="4" d157="4" d158="4" d159="4" d160="4" d161="4" d162="4" d163="4" d164="4" d165="4" d166="4" d167="4" d168="4" d169="4" d170="4" d171="4" d172="4" d173="4" d174="4" d175="4" d176="4" d177="4" d178="4" d179="4" d180="4" d181="4" d182="4" d183="4" d184="4" d185="4" d186="4" d187="4" d188="4" d189="4" d190="4" d191="4" d192="4" d193="4" d194="4" d195="4" d196="4" d197="4" d198="4" d199="4" d200="4" d201="4" d202="4" d203="4" d204="4" d205="4" d206="4" d207="4" d208="4" d209="4" d210="4" d211="4" d212="4" d213="4" d214="4" d215="4" d216="4" d217="4" d218="4" d219="4" d220="4" d221="4" d222="4" d223="4" d224="4" d225="4" d226="4" d227="4" d228="4" d229="4" d230="4" d231="4" d232="4" d233="4" d234="4" d235="4" d236="4" d237="4" d238="4" d239="4" d240="4" d241="4" d242="4" d243="4" d244="4" d245="4" d246="4" d247="4" d248="4" d249="4" d250="4" d251="4" d252="4" d253="4" d254="4" d255="4" d256="4" d257="4" d258="4" d259="4" d260="4" d261="4" d262="4" d263="4" d264="4" d265="4" d266="4" d267="4" d268="4" d269="4" d270="4" d271="4" d272="4" d273="4" d274="4" d275="4" d276="4" d277="4" d278="4" d279="4" d280="4" d281="4" d282="4" d283="4" d284="4" d285="4" d286="4" d287="4" d288="4" d289="4" d290="4" d291="4" d292="4" d293="4" d294="4" d295="4" d296="4" d297="4" d298="4" d299="4" d300="4" d301="4" d302="4" d303="4" d304="4" d305="4" d306="4" d307="4" d308="4" d309="4" d310="4" d311="4" d312="4" d313="4" d314="4" d315="4" d316="4" d317="4" d318="4" d319="4" d320="4" d321="4" d322="4" d323="4" d324="4" d325="4" d326="4" d327="4" d328="4" d329="4" d330="4" d331="4" d332="4" d333="4" d334="4" d335="4" d336="4" d337="4" d338="4" d339="4" d340="4" d341="4" d342="4" d343="4" d344="4" d345="4" d346="4" d347="4" d348="4" d349="4" d350="4" d351="4" d352="4" d353="4" d354="4" d355="4" d356="4" d357="4" d358="4" d359="4" d360="4" d361="4" d362="4" d363="4" d364="4" d365="4"/>\n' +
        '<OccupancyYearProfile id="5" d1="5" d2="5" d3="5" d4="5" d5="5" d6="5" d7="5" d8="5" d9="5" d10="5" d11="5" d12="5" d13="5" d14="5" d15="5" d16="5" d17="5" d18="5" d19="5" d20="5" d21="5" d22="5" d23="5" d24="5" d25="5" d26="5" d27="5" d28="5" d29="5" d30="5" d31="5" d32="5" d33="5" d34="5" d35="5" d36="5" d37="5" d38="5" d39="5" d40="5" d41="5" d42="5" d43="5" d44="5" d45="5" d46="5" d47="5" d48="5" d49="5" d50="5" d51="5" d52="5" d53="5" d54="5" d55="5" d56="5" d57="5" d58="5" d59="5" d60="5" d61="5" d62="5" d63="5" d64="5" d65="5" d66="5" d67="5" d68="5" d69="5" d70="5" d71="5" d72="5" d73="5" d74="5" d75="5" d76="5" d77="5" d78="5" d79="5" d80="5" d81="5" d82="5" d83="5" d84="5" d85="5" d86="5" d87="5" d88="5" d89="5" d90="5" d91="5" d92="5" d93="5" d94="5" d95="5" d96="5" d97="5" d98="5" d99="5" d100="5" d101="5" d102="5" d103="5" d104="5" d105="5" d106="5" d107="5" d108="5" d109="5" d110="5" d111="5" d112="5" d113="5" d114="5" d115="5" d116="5" d117="5" d118="5" d119="5" d120="5" d121="5" d122="5" d123="5" d124="5" d125="5" d126="5" d127="5" d128="5" d129="5" d130="5" d131="5" d132="5" d133="5" d134="5" d135="5" d136="5" d137="5" d138="5" d139="5" d140="5" d141="5" d142="5" d143="5" d144="5" d145="5" d146="5" d147="5" d148="5" d149="5" d150="5" d151="5" d152="5" d153="5" d154="5" d155="5" d156="5" d157="5" d158="5" d159="5" d160="5" d161="5" d162="5" d163="5" d164="5" d165="5" d166="5" d167="5" d168="5" d169="5" d170="5" d171="5" d172="5" d173="5" d174="5" d175="5" d176="5" d177="5" d178="5" d179="5" d180="5" d181="5" d182="5" d183="5" d184="5" d185="5" d186="5" d187="5" d188="5" d189="5" d190="5" d191="5" d192="5" d193="5" d194="5" d195="5" d196="5" d197="5" d198="5" d199="5" d200="5" d201="5" d202="5" d203="5" d204="5" d205="5" d206="5" d207="5" d208="5" d209="5" d210="5" d211="5" d212="5" d213="5" d214="5" d215="5" d216="5" d217="5" d218="5" d219="5" d220="5" d221="5" d222="5" d223="5" d224="5" d225="5" d226="5" d227="5" d228="5" d229="5" d230="5" d231="5" d232="5" d233="5" d234="5" d235="5" d236="5" d237="5" d238="5" d239="5" d240="5" d241="5" d242="5" d243="5" d244="5" d245="5" d246="5" d247="5" d248="5" d249="5" d250="5" d251="5" d252="5" d253="5" d254="5" d255="5" d256="5" d257="5" d258="5" d259="5" d260="5" d261="5" d262="5" d263="5" d264="5" d265="5" d266="5" d267="5" d268="5" d269="5" d270="5" d271="5" d272="5" d273="5" d274="5" d275="5" d276="5" d277="5" d278="5" d279="5" d280="5" d281="5" d282="5" d283="5" d284="5" d285="5" d286="5" d287="5" d288="5" d289="5" d290="5" d291="5" d292="5" d293="5" d294="5" d295="5" d296="5" d297="5" d298="5" d299="5" d300="5" d301="5" d302="5" d303="5" d304="5" d305="5" d306="5" d307="5" d308="5" d309="5" d310="5" d311="5" d312="5" d313="5" d314="5" d315="5" d316="5" d317="5" d318="5" d319="5" d320="5" d321="5" d322="5" d323="5" d324="5" d325="5" d326="5" d327="5" d328="5" d329="5" d330="5" d331="5" d332="5" d333="5" d334="5" d335="5" d336="5" d337="5" d338="5" d339="5" d340="5" d341="5" d342="5" d343="5" d344="5" d345="5" d346="5" d347="5" d348="5" d349="5" d350="5" d351="5" d352="5" d353="5" d354="5" d355="5" d356="5" d357="5" d358="5" d359="5" d360="5" d361="5" d362="5" d363="5" d364="5" d365="5"/>\n' +
        '<OccupancyYearProfile id="6" d1="6" d2="6" d3="6" d4="6" d5="6" d6="6" d7="6" d8="6" d9="6" d10="6" d11="6" d12="6" d13="6" d14="6" d15="6" d16="6" d17="6" d18="6" d19="6" d20="6" d21="6" d22="6" d23="6" d24="6" d25="6" d26="6" d27="6" d28="6" d29="6" d30="6" d31="6" d32="6" d33="6" d34="6" d35="6" d36="6" d37="6" d38="6" d39="6" d40="6" d41="6" d42="6" d43="6" d44="6" d45="6" d46="6" d47="6" d48="6" d49="6" d50="6" d51="6" d52="6" d53="6" d54="6" d55="6" d56="6" d57="6" d58="6" d59="6" d60="6" d61="6" d62="6" d63="6" d64="6" d65="6" d66="6" d67="6" d68="6" d69="6" d70="6" d71="6" d72="6" d73="6" d74="6" d75="6" d76="6" d77="6" d78="6" d79="6" d80="6" d81="6" d82="6" d83="6" d84="6" d85="6" d86="6" d87="6" d88="6" d89="6" d90="6" d91="6" d92="6" d93="6" d94="6" d95="6" d96="6" d97="6" d98="6" d99="6" d100="6" d101="6" d102="6" d103="6" d104="6" d105="6" d106="6" d107="6" d108="6" d109="6" d110="6" d111="6" d112="6" d113="6" d114="6" d115="6" d116="6" d117="6" d118="6" d119="6" d120="6" d121="6" d122="6" d123="6" d124="6" d125="6" d126="6" d127="6" d128="6" d129="6" d130="6" d131="6" d132="6" d133="6" d134="6" d135="6" d136="6" d137="6" d138="6" d139="6" d140="6" d141="6" d142="6" d143="6" d144="6" d145="6" d146="6" d147="6" d148="6" d149="6" d150="6" d151="6" d152="6" d153="6" d154="6" d155="6" d156="6" d157="6" d158="6" d159="6" d160="6" d161="6" d162="6" d163="6" d164="6" d165="6" d166="6" d167="6" d168="6" d169="6" d170="6" d171="6" d172="6" d173="6" d174="6" d175="6" d176="6" d177="6" d178="6" d179="6" d180="6" d181="6" d182="6" d183="6" d184="6" d185="6" d186="6" d187="6" d188="6" d189="6" d190="6" d191="6" d192="6" d193="6" d194="6" d195="6" d196="6" d197="6" d198="6" d199="6" d200="6" d201="6" d202="6" d203="6" d204="6" d205="6" d206="6" d207="6" d208="6" d209="6" d210="6" d211="6" d212="6" d213="6" d214="6" d215="6" d216="6" d217="6" d218="6" d219="6" d220="6" d221="6" d222="6" d223="6" d224="6" d225="6" d226="6" d227="6" d228="6" d229="6" d230="6" d231="6" d232="6" d233="6" d234="6" d235="6" d236="6" d237="6" d238="6" d239="6" d240="6" d241="6" d242="6" d243="6" d244="6" d245="6" d246="6" d247="6" d248="6" d249="6" d250="6" d251="6" d252="6" d253="6" d254="6" d255="6" d256="6" d257="6" d258="6" d259="6" d260="6" d261="6" d262="6" d263="6" d264="6" d265="6" d266="6" d267="6" d268="6" d269="6" d270="6" d271="6" d272="6" d273="6" d274="6" d275="6" d276="6" d277="6" d278="6" d279="6" d280="6" d281="6" d282="6" d283="6" d284="6" d285="6" d286="6" d287="6" d288="6" d289="6" d290="6" d291="6" d292="6" d293="6" d294="6" d295="6" d296="6" d297="6" d298="6" d299="6" d300="6" d301="6" d302="6" d303="6" d304="6" d305="6" d306="6" d307="6" d308="6" d309="6" d310="6" d311="6" d312="6" d313="6" d314="6" d315="6" d316="6" d317="6" d318="6" d319="6" d320="6" d321="6" d322="6" d323="6" d324="6" d325="6" d326="6" d327="6" d328="6" d329="6" d330="6" d331="6" d332="6" d333="6" d334="6" d335="6" d336="6" d337="6" d338="6" d339="6" d340="6" d341="6" d342="6" d343="6" d344="6" d345="6" d346="6" d347="6" d348="6" d349="6" d350="6" d351="6" d352="6" d353="6" d354="6" d355="6" d356="6" d357="6" d358="6" d359="6" d360="6" d361="6" d362="6" d363="6" d364="6" d365="6"/>\n' +
        '<OccupancyYearProfile id="7" d1="7" d2="7" d3="7" d4="7" d5="7" d6="7" d7="7" d8="7" d9="7" d10="7" d11="7" d12="7" d13="7" d14="7" d15="7" d16="7" d17="7" d18="7" d19="7" d20="7" d21="7" d22="7" d23="7" d24="7" d25="7" d26="7" d27="7" d28="7" d29="7" d30="7" d31="7" d32="7" d33="7" d34="7" d35="7" d36="7" d37="7" d38="7" d39="7" d40="7" d41="7" d42="7" d43="7" d44="7" d45="7" d46="7" d47="7" d48="7" d49="7" d50="7" d51="7" d52="7" d53="7" d54="7" d55="7" d56="7" d57="7" d58="7" d59="7" d60="7" d61="7" d62="7" d63="7" d64="7" d65="7" d66="7" d67="7" d68="7" d69="7" d70="7" d71="7" d72="7" d73="7" d74="7" d75="7" d76="7" d77="7" d78="7" d79="7" d80="7" d81="7" d82="7" d83="7" d84="7" d85="7" d86="7" d87="7" d88="7" d89="7" d90="7" d91="7" d92="7" d93="7" d94="7" d95="7" d96="7" d97="7" d98="7" d99="7" d100="7" d101="7" d102="7" d103="7" d104="7" d105="7" d106="7" d107="7" d108="7" d109="7" d110="7" d111="7" d112="7" d113="7" d114="7" d115="7" d116="7" d117="7" d118="7" d119="7" d120="7" d121="7" d122="7" d123="7" d124="7" d125="7" d126="7" d127="7" d128="7" d129="7" d130="7" d131="7" d132="7" d133="7" d134="7" d135="7" d136="7" d137="7" d138="7" d139="7" d140="7" d141="7" d142="7" d143="7" d144="7" d145="7" d146="7" d147="7" d148="7" d149="7" d150="7" d151="7" d152="7" d153="7" d154="7" d155="7" d156="7" d157="7" d158="7" d159="7" d160="7" d161="7" d162="7" d163="7" d164="7" d165="7" d166="7" d167="7" d168="7" d169="7" d170="7" d171="7" d172="7" d173="7" d174="7" d175="7" d176="7" d177="7" d178="7" d179="7" d180="7" d181="7" d182="7" d183="7" d184="7" d185="7" d186="7" d187="7" d188="7" d189="7" d190="7" d191="7" d192="7" d193="7" d194="7" d195="7" d196="7" d197="7" d198="7" d199="7" d200="7" d201="7" d202="7" d203="7" d204="7" d205="7" d206="7" d207="7" d208="7" d209="7" d210="7" d211="7" d212="7" d213="7" d214="7" d215="7" d216="7" d217="7" d218="7" d219="7" d220="7" d221="7" d222="7" d223="7" d224="7" d225="7" d226="7" d227="7" d228="7" d229="7" d230="7" d231="7" d232="7" d233="7" d234="7" d235="7" d236="7" d237="7" d238="7" d239="7" d240="7" d241="7" d242="7" d243="7" d244="7" d245="7" d246="7" d247="7" d248="7" d249="7" d250="7" d251="7" d252="7" d253="7" d254="7" d255="7" d256="7" d257="7" d258="7" d259="7" d260="7" d261="7" d262="7" d263="7" d264="7" d265="7" d266="7" d267="7" d268="7" d269="7" d270="7" d271="7" d272="7" d273="7" d274="7" d275="7" d276="7" d277="7" d278="7" d279="7" d280="7" d281="7" d282="7" d283="7" d284="7" d285="7" d286="7" d287="7" d288="7" d289="7" d290="7" d291="7" d292="7" d293="7" d294="7" d295="7" d296="7" d297="7" d298="7" d299="7" d300="7" d301="7" d302="7" d303="7" d304="7" d305="7" d306="7" d307="7" d308="7" d309="7" d310="7" d311="7" d312="7" d313="7" d314="7" d315="7" d316="7" d317="7" d318="7" d319="7" d320="7" d321="7" d322="7" d323="7" d324="7" d325="7" d326="7" d327="7" d328="7" d329="7" d330="7" d331="7" d332="7" d333="7" d334="7" d335="7" d336="7" d337="7" d338="7" d339="7" d340="7" d341="7" d342="7" d343="7" d344="7" d345="7" d346="7" d347="7" d348="7" d349="7" d350="7" d351="7" d352="7" d353="7" d354="7" d355="7" d356="7" d357="7" d358="7" d359="7" d360="7" d361="7" d362="7" d363="7" d364="7" d365="7"/>\n' +
        '<OccupancyYearProfile id="8" d1="8" d2="8" d3="8" d4="8" d5="8" d6="8" d7="8" d8="8" d9="8" d10="8" d11="8" d12="8" d13="8" d14="8" d15="8" d16="8" d17="8" d18="8" d19="8" d20="8" d21="8" d22="8" d23="8" d24="8" d25="8" d26="8" d27="8" d28="8" d29="8" d30="8" d31="8" d32="8" d33="8" d34="8" d35="8" d36="8" d37="8" d38="8" d39="8" d40="8" d41="8" d42="8" d43="8" d44="8" d45="8" d46="8" d47="8" d48="8" d49="8" d50="8" d51="8" d52="8" d53="8" d54="8" d55="8" d56="8" d57="8" d58="8" d59="8" d60="8" d61="8" d62="8" d63="8" d64="8" d65="8" d66="8" d67="8" d68="8" d69="8" d70="8" d71="8" d72="8" d73="8" d74="8" d75="8" d76="8" d77="8" d78="8" d79="8" d80="8" d81="8" d82="8" d83="8" d84="8" d85="8" d86="8" d87="8" d88="8" d89="8" d90="8" d91="8" d92="8" d93="8" d94="8" d95="8" d96="8" d97="8" d98="8" d99="8" d100="8" d101="8" d102="8" d103="8" d104="8" d105="8" d106="8" d107="8" d108="8" d109="8" d110="8" d111="8" d112="8" d113="8" d114="8" d115="8" d116="8" d117="8" d118="8" d119="8" d120="8" d121="8" d122="8" d123="8" d124="8" d125="8" d126="8" d127="8" d128="8" d129="8" d130="8" d131="8" d132="8" d133="8" d134="8" d135="8" d136="8" d137="8" d138="8" d139="8" d140="8" d141="8" d142="8" d143="8" d144="8" d145="8" d146="8" d147="8" d148="8" d149="8" d150="8" d151="8" d152="8" d153="8" d154="8" d155="8" d156="8" d157="8" d158="8" d159="8" d160="8" d161="8" d162="8" d163="8" d164="8" d165="8" d166="8" d167="8" d168="8" d169="8" d170="8" d171="8" d172="8" d173="8" d174="8" d175="8" d176="8" d177="8" d178="8" d179="8" d180="8" d181="8" d182="8" d183="8" d184="8" d185="8" d186="8" d187="8" d188="8" d189="8" d190="8" d191="8" d192="8" d193="8" d194="8" d195="8" d196="8" d197="8" d198="8" d199="8" d200="8" d201="8" d202="8" d203="8" d204="8" d205="8" d206="8" d207="8" d208="8" d209="8" d210="8" d211="8" d212="8" d213="8" d214="8" d215="8" d216="8" d217="8" d218="8" d219="8" d220="8" d221="8" d222="8" d223="8" d224="8" d225="8" d226="8" d227="8" d228="8" d229="8" d230="8" d231="8" d232="8" d233="8" d234="8" d235="8" d236="8" d237="8" d238="8" d239="8" d240="8" d241="8" d242="8" d243="8" d244="8" d245="8" d246="8" d247="8" d248="8" d249="8" d250="8" d251="8" d252="8" d253="8" d254="8" d255="8" d256="8" d257="8" d258="8" d259="8" d260="8" d261="8" d262="8" d263="8" d264="8" d265="8" d266="8" d267="8" d268="8" d269="8" d270="8" d271="8" d272="8" d273="8" d274="8" d275="8" d276="8" d277="8" d278="8" d279="8" d280="8" d281="8" d282="8" d283="8" d284="8" d285="8" d286="8" d287="8" d288="8" d289="8" d290="8" d291="8" d292="8" d293="8" d294="8" d295="8" d296="8" d297="8" d298="8" d299="8" d300="8" d301="8" d302="8" d303="8" d304="8" d305="8" d306="8" d307="8" d308="8" d309="8" d310="8" d311="8" d312="8" d313="8" d314="8" d315="8" d316="8" d317="8" d318="8" d319="8" d320="8" d321="8" d322="8" d323="8" d324="8" d325="8" d326="8" d327="8" d328="8" d329="8" d330="8" d331="8" d332="8" d333="8" d334="8" d335="8" d336="8" d337="8" d338="8" d339="8" d340="8" d341="8" d342="8" d343="8" d344="8" d345="8" d346="8" d347="8" d348="8" d349="8" d350="8" d351="8" d352="8" d353="8" d354="8" d355="8" d356="8" d357="8" d358="8" d359="8" d360="8" d361="8" d362="8" d363="8" d364="8" d365="8"/>\n' +
        '<OccupancyYearProfile id="9" d1="9" d2="9" d3="9" d4="9" d5="9" d6="9" d7="9" d8="9" d9="9" d10="9" d11="9" d12="9" d13="9" d14="9" d15="9" d16="9" d17="9" d18="9" d19="9" d20="9" d21="9" d22="9" d23="9" d24="9" d25="9" d26="9" d27="9" d28="9" d29="9" d30="9" d31="9" d32="9" d33="9" d34="9" d35="9" d36="9" d37="9" d38="9" d39="9" d40="9" d41="9" d42="9" d43="9" d44="9" d45="9" d46="9" d47="9" d48="9" d49="9" d50="9" d51="9" d52="9" d53="9" d54="9" d55="9" d56="9" d57="9" d58="9" d59="9" d60="9" d61="9" d62="9" d63="9" d64="9" d65="9" d66="9" d67="9" d68="9" d69="9" d70="9" d71="9" d72="9" d73="9" d74="9" d75="9" d76="9" d77="9" d78="9" d79="9" d80="9" d81="9" d82="9" d83="9" d84="9" d85="9" d86="9" d87="9" d88="9" d89="9" d90="9" d91="9" d92="9" d93="9" d94="9" d95="9" d96="9" d97="9" d98="9" d99="9" d100="9" d101="9" d102="9" d103="9" d104="9" d105="9" d106="9" d107="9" d108="9" d109="9" d110="9" d111="9" d112="9" d113="9" d114="9" d115="9" d116="9" d117="9" d118="9" d119="9" d120="9" d121="9" d122="9" d123="9" d124="9" d125="9" d126="9" d127="9" d128="9" d129="9" d130="9" d131="9" d132="9" d133="9" d134="9" d135="9" d136="9" d137="9" d138="9" d139="9" d140="9" d141="9" d142="9" d143="9" d144="9" d145="9" d146="9" d147="9" d148="9" d149="9" d150="9" d151="9" d152="9" d153="9" d154="9" d155="9" d156="9" d157="9" d158="9" d159="9" d160="9" d161="9" d162="9" d163="9" d164="9" d165="9" d166="9" d167="9" d168="9" d169="9" d170="9" d171="9" d172="9" d173="9" d174="9" d175="9" d176="9" d177="9" d178="9" d179="9" d180="9" d181="9" d182="9" d183="9" d184="9" d185="9" d186="9" d187="9" d188="9" d189="9" d190="9" d191="9" d192="9" d193="9" d194="9" d195="9" d196="9" d197="9" d198="9" d199="9" d200="9" d201="9" d202="9" d203="9" d204="9" d205="9" d206="9" d207="9" d208="9" d209="9" d210="9" d211="9" d212="9" d213="9" d214="9" d215="9" d216="9" d217="9" d218="9" d219="9" d220="9" d221="9" d222="9" d223="9" d224="9" d225="9" d226="9" d227="9" d228="9" d229="9" d230="9" d231="9" d232="9" d233="9" d234="9" d235="9" d236="9" d237="9" d238="9" d239="9" d240="9" d241="9" d242="9" d243="9" d244="9" d245="9" d246="9" d247="9" d248="9" d249="9" d250="9" d251="9" d252="9" d253="9" d254="9" d255="9" d256="9" d257="9" d258="9" d259="9" d260="9" d261="9" d262="9" d263="9" d264="9" d265="9" d266="9" d267="9" d268="9" d269="9" d270="9" d271="9" d272="9" d273="9" d274="9" d275="9" d276="9" d277="9" d278="9" d279="9" d280="9" d281="9" d282="9" d283="9" d284="9" d285="9" d286="9" d287="9" d288="9" d289="9" d290="9" d291="9" d292="9" d293="9" d294="9" d295="9" d296="9" d297="9" d298="9" d299="9" d300="9" d301="9" d302="9" d303="9" d304="9" d305="9" d306="9" d307="9" d308="9" d309="9" d310="9" d311="9" d312="9" d313="9" d314="9" d315="9" d316="9" d317="9" d318="9" d319="9" d320="9" d321="9" d322="9" d323="9" d324="9" d325="9" d326="9" d327="9" d328="9" d329="9" d330="9" d331="9" d332="9" d333="9" d334="9" d335="9" d336="9" d337="9" d338="9" d339="9" d340="9" d341="9" d342="9" d343="9" d344="9" d345="9" d346="9" d347="9" d348="9" d349="9" d350="9" d351="9" d352="9" d353="9" d354="9" d355="9" d356="9" d357="9" d358="9" d359="9" d360="9" d361="9" d362="9" d363="9" d364="9" d365="9"/>\n' +
        '<OccupancyYearProfile id="10" d1="10" d2="10" d3="10" d4="10" d5="10" d6="10" d7="10" d8="10" d9="10" d10="10" d11="10" d12="10" d13="10" d14="10" d15="10" d16="10" d17="10" d18="10" d19="10" d20="10" d21="10" d22="10" d23="10" d24="10" d25="10" d26="10" d27="10" d28="10" d29="10" d30="10" d31="10" d32="10" d33="10" d34="10" d35="10" d36="10" d37="10" d38="10" d39="10" d40="10" d41="10" d42="10" d43="10" d44="10" d45="10" d46="10" d47="10" d48="10" d49="10" d50="10" d51="10" d52="10" d53="10" d54="10" d55="10" d56="10" d57="10" d58="10" d59="10" d60="10" d61="10" d62="10" d63="10" d64="10" d65="10" d66="10" d67="10" d68="10" d69="10" d70="10" d71="10" d72="10" d73="10" d74="10" d75="10" d76="10" d77="10" d78="10" d79="10" d80="10" d81="10" d82="10" d83="10" d84="10" d85="10" d86="10" d87="10" d88="10" d89="10" d90="10" d91="10" d92="10" d93="10" d94="10" d95="10" d96="10" d97="10" d98="10" d99="10" d100="10" d101="10" d102="10" d103="10" d104="10" d105="10" d106="10" d107="10" d108="10" d109="10" d110="10" d111="10" d112="10" d113="10" d114="10" d115="10" d116="10" d117="10" d118="10" d119="10" d120="10" d121="10" d122="10" d123="10" d124="10" d125="10" d126="10" d127="10" d128="10" d129="10" d130="10" d131="10" d132="10" d133="10" d134="10" d135="10" d136="10" d137="10" d138="10" d139="10" d140="10" d141="10" d142="10" d143="10" d144="10" d145="10" d146="10" d147="10" d148="10" d149="10" d150="10" d151="10" d152="10" d153="10" d154="10" d155="10" d156="10" d157="10" d158="10" d159="10" d160="10" d161="10" d162="10" d163="10" d164="10" d165="10" d166="10" d167="10" d168="10" d169="10" d170="10" d171="10" d172="10" d173="10" d174="10" d175="10" d176="10" d177="10" d178="10" d179="10" d180="10" d181="10" d182="10" d183="10" d184="10" d185="10" d186="10" d187="10" d188="10" d189="10" d190="10" d191="10" d192="10" d193="10" d194="10" d195="10" d196="10" d197="10" d198="10" d199="10" d200="10" d201="10" d202="10" d203="10" d204="10" d205="10" d206="10" d207="10" d208="10" d209="10" d210="10" d211="10" d212="10" d213="10" d214="10" d215="10" d216="10" d217="10" d218="10" d219="10" d220="10" d221="10" d222="10" d223="10" d224="10" d225="10" d226="10" d227="10" d228="10" d229="10" d230="10" d231="10" d232="10" d233="10" d234="10" d235="10" d236="10" d237="10" d238="10" d239="10" d240="10" d241="10" d242="10" d243="10" d244="10" d245="10" d246="10" d247="10" d248="10" d249="10" d250="10" d251="10" d252="10" d253="10" d254="10" d255="10" d256="10" d257="10" d258="10" d259="10" d260="10" d261="10" d262="10" d263="10" d264="10" d265="10" d266="10" d267="10" d268="10" d269="10" d270="10" d271="10" d272="10" d273="10" d274="10" d275="10" d276="10" d277="10" d278="10" d279="10" d280="10" d281="10" d282="10" d283="10" d284="10" d285="10" d286="10" d287="10" d288="10" d289="10" d290="10" d291="10" d292="10" d293="10" d294="10" d295="10" d296="10" d297="10" d298="10" d299="10" d300="10" d301="10" d302="10" d303="10" d304="10" d305="10" d306="10" d307="10" d308="10" d309="10" d310="10" d311="10" d312="10" d313="10" d314="10" d315="10" d316="10" d317="10" d318="10" d319="10" d320="10" d321="10" d322="10" d323="10" d324="10" d325="10" d326="10" d327="10" d328="10" d329="10" d330="10" d331="10" d332="10" d333="10" d334="10" d335="10" d336="10" d337="10" d338="10" d339="10" d340="10" d341="10" d342="10" d343="10" d344="10" d345="10" d346="10" d347="10" d348="10" d349="10" d350="10" d351="10" d352="10" d353="10" d354="10" d355="10" d356="10" d357="10" d358="10" d359="10" d360="10" d361="10" d362="10" d363="10" d364="10" d365="10"/>\n',
        file=open(xml_name, "a"))

## Final xml 

In [ ]:
# Multi-process print of the buildings in the XML
print("Printing the buildings on the XML file...")
import geometry_processor_db_2021

text = geometry_processor_db_2021.bx_run(buildings_df, final_envelope, occupancy_df)
print(text, file=open(xml_name, "a"))

print("</District>", file=open(xml_name, "a"))
print("</CitySim>", file=open(xml_name, "a"))

print("\n\n\n")

## Calling CitySim & simulation

In [ ]:
# Call CitySim ############################################################
print("Waking up CitySim...\n")
    
try:
    p = subprocess.Popen(['CitySim', xml_name], stderr=subprocess.PIPE)
    print("The simulation has begun!\n")
    # Print what CitySim solver is showing
    for line in p.stderr: 
        print(line.decode()) 
        if line.decode() == 'Simulation ended.\r\n':
            print("Results are being written...")

except Exception:
    print("Couldn't wake up CitySim. Please, check if CitySim solver is in the script's directory")
    time.sleep(5)
    sys.exit()

# Read results & store in DB

In [ ]:
# Read the results 
out_df = pd.read_csv('TH.tsv file',  sep='\t') # change file name
out_df

In [ ]:
time_series = {"egid": [],"values_array": [] ,"time_array": []}
time_series = pd.DataFrame(time_series)

time_series

In [ ]:
timestep_array = pd.date_range("2019-01-01 00:00", "2019-12-31 23:00", freq="60min").to_pydatetime().tolist()
timestep_array

In [ ]:
#Extracting simulated heating demand from CitySim results

egidset = pd.Series(buildings_df["egid"]) #105 egids
heating_string = 'Heating(Wh)'


for n in range(len(out_df.columns)): # We are only interested in heating demand
    for e in egidset:
        if str(e) in out_df.columns[n]:
            if heating_string in out_df.columns[n]:
                print("\nExtract results for building " + str(e))
                to_append = {"egid": e, "values_array": out_df.iloc[:,n].values.tolist(), "time_array": timestep_array}
                time_series = time_series.append(to_append, ignore_index=True)

    
time_series = time_series.drop_duplicates(["egid"])
time_series

In [ ]:
# Add gmlid to time_series

merged_egid_intersect = pd.read_sql_query("SELECT gmlid, egid FROM city.merged_egid_intersect", db_engine_2021)
time_series = pd.merge(time_series, merged_egid_intersect, on="egid", how="left").dropna()
time_series

In [ ]:
# Create table nrg8_time_series to insert in DB

nrg8_time_series = {"id": [], "objectclass_id": [],"gmlid": [] ,"values_unit": [] ,"time_interval": [] , "time_interval_unit": [], "acquisition_method":[], "interpolation_type":[]}
nrg8_time_series = pd.DataFrame(nrg8_time_series)
nrg8_time_series["gmlid"] = time_series["gmlid"]
nrg8_time_series = nrg8_time_series.merge(time_series, on="gmlid", how="left")

row_nbr_start = pd.read_sql_query("SELECT COUNT(id) FROM citydb.nrg8_time_series", db_engine_2021) #Last line in current table in DB
row_nbr = len(nrg8_time_series["gmlid"]) 
row_nbr_stop = row_nbr_start + 1 + row_nbr 

nrg8_time_series["objectclass_id"] = pd.DataFrame(np.repeat(202, row_nbr)) #regular time series
nrg8_time_series["id"] = pd.DataFrame(np.arange(row_nbr_start.iloc[0,0] + 1, row_nbr_stop.iloc[0,0])) 
nrg8_time_series["acquisition_method"] = pd.DataFrame(np.repeat('Simulation', row_nbr))
nrg8_time_series["interpolation_type"] = pd.DataFrame(np.repeat('AverageInSucceedingInterval', row_nbr)) #to change

nrg8_time_series["values_unit"] = pd.DataFrame(np.repeat('Wh', row_nbr))
nrg8_time_series["time_interval"] = pd.DataFrame(np.repeat('1', row_nbr))
nrg8_time_series["time_interval_unit"] = pd.DataFrame(np.repeat('h', row_nbr))

nrg8_time_series = nrg8_time_series.drop(["egid"], axis=1)
nrg8_time_series

In [ ]:
## Insert into the DB

nrg8_time_series.to_sql(name='nrg8_time_series', con = db_engine_2021, if_exists='append', index=False)

# Yearly total values from CitySim simulation

In [ ]:
yearly_values = {"egid": [],"values_array": []}
yearly_values = pd.DataFrame(yearly_values)
yearly_values

In [ ]:
#Extracting simulated heating demand from CitySim results --> summing each column to get yearly value

egidset = pd.Series(buildings_df["egid"]) #105 egids
heating_string = 'Heating(Wh)'

for n in range(len(out_df.columns)): # We are only interested in heating demand
    for e in egidset:
        if str(e) in out_df.columns[n]:
            if heating_string in out_df.columns[n]:
                print("\nExtract results for building " + str(e))
                to_append = {"egid": e, "values_array": out_df.iloc[:,n].sum()}
                yearly_values = yearly_values.append(to_append, ignore_index=True)
            
        
yearly_values = yearly_values.drop_duplicates(["egid"])
yearly_values

In [ ]:
# Add gmlid to yearly_values

merged_egid_intersect = pd.read_sql_query("SELECT gmlid, egid FROM city.merged_egid_intersect", db_engine_2021)
yearly_values = pd.merge(yearly_values, merged_egid_intersect, on="egid", how="left").dropna()
yearly_values

In [ ]:
# Convert energy into array type
list_values = list(map(lambda el:[el], yearly_values.iloc[:,1].values.tolist()))
energy_list = []
energy_list.append(list_values)
energy_list = pd.DataFrame(energy_list).transpose()
energy_list

In [ ]:
# Create table to add to annual_values for nrg8_time_series

annual_values =  {"gmlid": []}
annual_values = pd.DataFrame(annual_values)
annual_values["gmlid"] = yearly_values["gmlid"]
annual_values = annual_values.reset_index()

row_nbr_start = pd.read_sql_query("SELECT COUNT(id) FROM citydb.nrg8_time_series", db_engine_2021) #Last line in current table in DB
row_nbr = len(annual_values["gmlid"]) 
row_nbr_stop = row_nbr_start + 1 + row_nbr 

annual_values['objectclass_id'] = pd.DataFrame(np.repeat(202, row_nbr))
annual_values["id"] = pd.DataFrame(np.arange(row_nbr_start.iloc[0,0] + 1, row_nbr_stop.iloc[0,0])) 
annual_values["acquisition_method"] = pd.DataFrame(np.repeat('Simulation', row_nbr))

annual_values["values_array"] = pd.DataFrame(energy_list)
annual_values["values_unit"] = pd.DataFrame(np.repeat('W', row_nbr)) #check if this value is correct 
annual_values["time_interval"] = pd.DataFrame(np.repeat(1, row_nbr))
annual_values["time_interval_unit"] = pd.DataFrame(np.repeat('year', row_nbr))
annual_values["interpolation_type"] = pd.DataFrame(np.repeat('PrecedingTotal', row_nbr)) #to check

annual_values = annual_values.drop(["index"], axis=1)
annual_values

In [ ]:
# Insert into DB
annual_values.to_sql(name='nrg8_time_series', con = db_engine_2021, if_exists='append', index=False)